### 1. raw data import ###

In [1]:
# package import
import pandas as pd
import numpy as np
import string
from datetime import datetime, timedelta
import pytz
# import oracledb
# oracledb.init_oracle_client()

In [ ]:
# # LOT_NO별 통과공정 데이터 수집위한 쿼리 정의(제소구분 1, 2일 경우)
# search_start_date = '2023-01-01'
# search_end_date = '2024-12-31'

# query_data = f"""
# SELECT  A.LOT_NO                    AS  LOT_NO
#     ,   A.COMPANY_STEEL_GRADE       AS  사내강종
#     ,   A.COMPANY_STEEL_GRADE_NAME  AS  사내강종명
#     ,   B.DIRECTION                 AS  향선
#     ,   A.USE                       AS  용도
#     ,   C.ITM_KIND                  AS  품종
#     ,   A.SHAPE                     AS  주문형상
#     ,   A.HEAT_TREAT                AS  열처리
#     ,   A.SURFACE                   AS  표면
#     ,   TO_CHAR(C.HZTL, 'FM9990.000')   AS  설계외경
#     ,   C.VTCL                      AS  설계폭
#     ,   (SELECT '(' || C.LTH_TY || ')' || CODE_NAME FROM BESTERP.CM_CODEDETAIL WHERE CODE_TYPE = 'ST18' AND CODE = C.LTH_TY)
#                                     AS  주문길이_구분
#     ,   I.EXCH_LTH_MIN              AS  주문길이_최소길이
#     ,   I.EXCH_LTH_MAX              AS  주문길이_최대길이
#     ,   I.PROD_LTH_MULT             AS  주문길이_배척길이    
#     ,   A.FIRST_INPUT_TYPE          AS  투입구분
#     ,   A.FIRST_INPUT_SHAPE         AS  최초투입형상
#     ,   A.HEAT_NO                   AS  HEAT_NO
#     ,   A.FIRST_INPUT_MATERIAL_CODE AS  재료코드
#     ,   A.FIRST_INPUT_ED            AS  소재_외경
#     ,   A.FIRST_INPUT_WIDTH         AS  소재_폭
#     ,   A.FIRST_INPUT_QTY           AS  소재_수량
#     ,   A.FIRST_INPUT_WGT           AS  소재_중량
#     ,   A.FIRST_INPUT_LENGTH_MIN    AS  소재_소재길이
#     ,   Z.PROCESS_RANK              AS  공정순위
#     ,   Z.PROCESS_CODE              AS  공정코드
#     ,   NVL((SELECT MAIN_EQUIP_NM FROM CMES.SPRA1001 WHERE RTN_CODE = NVL(Z.PROCESS_CODE, Z.EQUIP_CODE)), Z.EQUIP_CODE)
#                                     AS  공정명
#     ,   Z.INPUT_QTY                 AS  투입내역_수량
#     ,   Z.INPUT_WGT                 AS  투입내역_중량
#     ,   Z.INPUT_HEAT_TREAT          AS  투입정보_열처리
#     ,   Z.INPUT_SURFACE             AS  투입정보_표면
#     ,   Z.INPUT_GR_KIND             AS  투입정보_GR종류
#     ,   Z.INPUT_SHAPE               AS  투입정보_형상
#     ,   Z.INPUT_ED                  AS  투입정보_외경
#     ,   Z.INPUT_WIDTH               AS  투입정보_폭
#     ,   Z.INPUT_LENGTH_MAX          AS  투입정보_길이
#     ,   Z.ITEM_ACCOUNT              AS  제소구분
#     ,   A.PRODUCTION_REQUEST_NO     AS  생산의뢰번호
#     ,   A.CUST_CODE                 AS  수요가코드
#     ,   CMES.SF_CUSTOMER_NAME(A.CUST_CODE)  AS  수요가명
# FROM    CMES.SPRC02002 A 
#     LEFT OUTER JOIN BESTERP.SSAZ4100 B ON A.ORDER_NO = B.CO_NO
#     LEFT OUTER JOIN BESTERP.SSAZ4110 C ON A.ORDER_NO = C.CO_NO AND A.ORDER_SEQ = C.CO_SER
#     LEFT OUTER JOIN CMES.SPRA1101 I ON A.PRODUCTION_REQUEST_NO = I.PREQ_NO
#     LEFT OUTER JOIN CMES.SPRC02003 Z ON A.LOT_NO = Z.LOT_NO
# WHERE   A.LOT_STATUS = '3'
# AND     (A.CLOSE_DATE >= TO_DATE('{search_start_date}','YYYY-MM-DD') AND A.CLOSE_DATE <= TO_DATE('{search_end_date}','YYYY-MM-DD'))    --종결일 기준
# """

In [ ]:
# with oracledb.connect(user="cmes_reader", password="cmes#40100", dsn="172.17.40.185:1521/BESTTEST") as conn:
#     with conn.cursor() as curr:
        
#         result = curr.execute(query_data)
#         columns = [column[0] for column in curr.description]        
#         data = result.fetchall()

#         # 데이터를 DataFrame으로 변환
#         query_data = pd.DataFrame(data, columns=columns)

# # 중복된 row데이터 삭제
# query_data.drop_duplicates(inplace=True)

# # 확인되는 row/columns 갯수를 400개 이하로 제한
# pd.set_option('display.max_row', 400)
# pd.set_option('display.max_columns', 400)
# pd.set_option('display.max_seq_items', None)

# # 결과 확인
# query_data

In [2]:
# Raw데이터(parquet file) import
query_data = pd.read_parquet('240605_process_design.parquet')

# Display에서 조회되는 row/columns 갯수를 400개 이하로 제한
# pd.set_option('display.max_row', 400)
pd.set_option('display.max_columns', 400)
pd.set_option('display.max_seq_items', None)

# 결과 확인
query_data

,USE_YN,IRN_NAME,IRN_CODE,ORD_IRN_STD,SHAP_TY,SURF_STAT,USE_TY,DETL_USE_TY,DETL_USE_TY_NAME,USE_GRD,APPL_CUST_NAME,APPL_CUST,CMPY_KIND,MATR_LARG,SZ_RNG_V_FR,SZ_RNG_V_TO,SZ_RNG_H_FR,SZ_RNG_H_TO,STD_ITM_CODE,DSIGN_CON_CODE,SIZE_TOL_STD,HEAT_PROC_YN,ORD_IRN_NAME,HZTL_MIN,HZTL_MAX,VTCL_MIN,VTCL_MAX,LTH_MIN,LTH_MAX,APPL_TY,TOL_APPL_TY,HZTL_TOL_MIN,HZTL_TOL_MAX,VTCL_TOL_MIN,VTCL_TOL_MAX,LTH_TOL_MIN,LTH_TOL_MAX,HEAT_PATRN,HEAT_PATRN_NAME,RTN_FIX,PRODC_DSIGN_NO,ITM_KIND,SPEC_REFN,LTH_RNG_FR,LTH_RNG_TO,STD_ITM_CODE.1,DSIGN_CON_CODE.1,DET_SEQ,RTN,PASS_RTN,RTN_SEQ,RTN_CODE,NEED_YN,PROD_HT_PATRN,GR_KIND,GR_MESH,GR_POINTING,EQUIP_NM,PLNT_CODE,MESSAGE_YN,GR_H,PROD_HT_SEQ,SPRB_PASS_RTN,SPRB_PASS_RTN_NM,MATR_PASS_RTN,MATR_PASS_RTN_NM,PROD_PASS_RTN,PROD_PASS_RTN_NM
0,Y,A36,AAA292,U3,RB,BS,Y,MP,기계산업 플랜트용,1,(주)마산특수강,21486,유통,B,85.001,100.0,85.001,100.0,BAAA292U3RBBSY1N,BS21486001,비율입력,N,A36,85.001,100.0,85.001,100.0,3000.0,7500.0,S,R,-1.5,1.5,-1.5,1.5,0.0,40.0,None,None,NaN,C201601078600,B,N,3000.0,7500.0,BAAA292U3RBBSY1N,BS21486001,1,MUN01BUN22PU124,PU124,3124080,BW90,N,None,None,NaN,NaN,소형정정-입고검사공통,B,None,None,NaN,MUN01,신제강 60톤 LF-VD-연주,BUN22,대형분괴소재-SBM 연마,PU124,압연봉강(BS/NH/85~100)
1,Y,A36,AAA292,U3,RB,BS,Y,MP,기계산업 플랜트용,1,(주)마산특수강,21486,유통,B,85.001,100.0,85.001,100.0,BAAA292U3RBBSY1N,BS21486001,비율입력,N,A36,85.001,100.0,85.001,100.0,3000.0,7500.0,S,R,-1.5,1.5,-1.5,1.5,0.0,40.0,None,None,NaN,C201601078600,B,N,3000.0,7500.0,BAAA292U3RBBSY1N,BS21486001,1,MUN01BUN22PU124,PU124,3250220,BX00,N,None,None,NaN,NaN,소형정정-포장공통,B,None,None,NaN,MUN01,신제강 60톤 LF-VD-연주,BUN22,대형분괴소재-SBM 연마,PU124,압연봉강(BS/NH/85~100)
2,Y,A36,AAA292,U3,RB,BS,Y,MP,기계산업 플랜트용,1,(주)마산특수강,21486,유통,B,85.001,100.0,85.001,100.0,BAAA292U3RBBSY1N,BS21486001,비율입력,N,A36,85.001,100.0,85.001,100.0,3000.0,7500.0,S,R,-1.5,1.5,-1.5,1.5,0.0,40.0,None,None,NaN,C201601078600,B,N,3000.0,7500.0,BAAA292U3RBBSY1N,BS21486001,1,MUN01BUN22PU124,PU124,3364350,BY01,N,None,None,NaN,NaN,소형정정-입고,B,None,None,NaN,MUN01,신제강 60톤 LF-VD-연주,BUN22,대형분괴소재-SBM 연마,PU124,압연봉강(BS/NH/85~100)
3,Y,A36,AAA292,U3,RB,BS,Y,MP,기계산업 플랜트용,1,(주)마산특수강,21486,유통,B,85.001,100.0,85.001,100.0,BAAA292U3RBBSY1N,BS21486001,비율입력,N,A36,85.001,100.0,85.001,100.0,3000.0,7500.0,S,R,-1.5,1.5,-1.5,1.5,0.0,40.0,None,None,NaN,C201601078600,B,N,3000.0,7500.0,BAAA292U3RBBSY1N,BS21486001,1,MUN01BUN22PU124,MUN01,10,G101,N,None,None,NaN,NaN,3제강-60톤 전기로,G,None,None,NaN,MUN01,신제강 60톤 LF-VD-연주,BUN22,대형분괴소재-SBM 연마,PU124,압연봉강(BS/NH/85~100)
4,Y,A36,AAA292,U3,RB,BS,Y,MP,기계산업 플랜트용,1,(주)마산특수강,21486,유통,B,85.001,100.0,85.001,100.0,BAAA292U3RBBSY1N,BS21486001,비율입력,N,A36,85.001,100.0,85.001,100.0,3000.0,7500.0,S,R,-1.5,1.5,-1.5,1.5,0.0,40.0,None,None,NaN,C201601078600,B,N,3000.0,7500.0,BAAA292U3RBBSY1N,BS21486001,1,MUN01BUN22PU124,MUN01,203370,G201,N,None,None,NaN,NaN,3제강-60톤 LF,G,None,None,NaN,MUN01,신제강 60톤 LF-VD-연주,BUN22,대형분괴소재-SBM 연마,PU124,압연봉강(BS/NH/85~100)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364182,Y,STEL12PM,VZA001,ZZ,IG,BS,Y,RM,재용해용,1,CSS특수합금연구그룹,Z0039,None,I,85.000,200.0,85.000,200.0,ZVZA001ZZIGBSY1N,ISZ0039001,수치입력,N,STELLITE12,80.000,250.0,80.000,250.0,0.0,9999.0,S,N,-1.0,1.0,-1.0,1.0,0.0,40.0,None,None,NaN,R2018106094000,Z,I,0.0,9999.0,ZVZA001ZZIGBSY1N,ISZ0039001,1,MT005PT342,PT342,1094750,3Y01,N,None,None,NaN,NaN,3제강-특제-입고,3,None,None,NaN,MT005,VIM 25 KG,None,None,PT342,VIM 강괴 입고
364183,Y,STEL12PM,VZA001,ZZ,IG,BS,Y,RM,재용해용,1,CSS특수합금연구그룹,Z0039,None,I,85.000,200.0,85.000,200.0,ZVZA001ZZIGBSY1N,ISZ0039001,수치입력,N,STELLITE12,80.000,250.0,80.000,250.0,0.0,9999.0,S,N,-1.0,1.0,-1.0,1.0,0.0,40.0,None,None,NaN,R2018106094000,Z,I,0.0,9999.0,ZVZA001ZZIGBSY1N,ISZ0039001,2,MS904PS606,PS606,1094750,3Y01,N,None,None,NaN,NaN,3제강-특제-입고,3,None,None,NaN,MS904,25KG VIM,None,None,PS606,VIM강괴입고
364184,Y,STEL12PM,VZA001,ZZ,IG,BS,Y,RM,재용해용,1,CSS특수합금연구

In [4]:
# 특정컬럼의 고유값을 unique_values에 저장
unique_values = query_data['LOT_NO'].unique()

# unique_values의 길이 구하기
length = len(unique_values)

# 결과 확인
print(length)

298552


In [5]:
# 특정컬럼의 고유값을 unique_values에 저장
unique_values = query_data['생산의뢰번호'].unique()

# unique_values의 길이 구하기
length = len(unique_values)

# 결과 확인
print(length)

62686


### 2. 파생변수 생성 및 1차 데이터 전처리 ###

In [6]:
# 데이터프레임 기초정보 확인
query_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2814258 entries, 0 to 2814257
Data columns (total 40 columns):
 #   Column     Dtype  
---  ------     -----  
 0   LOT_NO     object 
 1   사내강종       object 
 2   사내강종명      object 
 3   향선         object 
 4   용도         object 
 5   품종         object 
 6   주문형상       object 
 7   열처리        object 
 8   표면         object 
 9   설계외경       object 
 10  설계폭        float64
 11  주문길이_구분    object 
 12  주문길이_최소길이  int64  
 13  주문길이_최대길이  int64  
 14  주문길이_배척길이  float64
 15  투입구분       object 
 16  최초투입형상     object 
 17  HEAT_NO    object 
 18  재료코드       object 
 19  소재_외경      float64
 20  소재_폭       float64
 21  소재_수량      float64
 22  소재_중량      int64  
 23  소재_소재길이    int64  
 24  공정순위       int64  
 25  공정코드       object 
 26  공정명        object 
 27  투입내역_수량    float64
 28  투입내역_중량    float64
 29  투입정보_열처리   object 
 30  투입정보_표면    object 
 31  투입정보_GR종류  object 
 32  투입정보_형상    object 
 33  투입정보_외경    float64
 34  투입정보_폭     float64

In [7]:
# 특정 컬럼의 데이터 class를 object에서 float64로 변환
query_data['설계외경'] = query_data['설계외경'].astype(float)

In [8]:
# 특정 3개 이상의 컬럼명을 지정하여 해당 컬럼 데이터를 삭제
columns_to_drop = ['향선','용도','주문길이_구분','주문길이_최소길이','주문길이_최대길이','주문길이_배척길이',
                   '투입구분','소재_수량','소재_중량','소재_소재길이',
                   '투입내역_수량','투입내역_중량','투입정보_열처리','투입정보_표면','투입정보_GR종류',
                   '투입정보_길이','수요가코드','수요가명']
query_data = query_data.drop(columns=columns_to_drop)

# 결과 확인
query_data

,LOT_NO,사내강종,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,재료코드,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호
0,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E22S04W0360
1,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E22S04W0360
2,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E22S04W0360
3,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E22S04W0360
4,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E22S04W0360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814253,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D24S05W0269
2814254,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D24S05W0269
2814255,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D24S05W0269
2814256,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D24S05W0269


In [9]:
# 생산의뢰번호별 설계key 데이터 import
design_code = pd.read_parquet('process_design.parquet')

# 결과 확인
design_code

,PREQ_NO,수주년월,STD_ITM_CODE,DSIGN_CON_CODE
0,D20N05W0002,202101,NSSF006U1DCBSY1Y,IS21464001
1,D20F07W0580,202101,FSMF00800RBRTY1Y,IC00000001
2,D20F08W0322,202101,FTLE00200FBBSY1Y,IS25469001
3,D20F06W0539,202101,FSMA06100RBRTY1Y,IS21464002
4,D20F07W0480,202101,FTLE00200FBMDY1Y,IS25469010
...,...,...,...,...
185028,E24W06W0287,202406,WSSH03003WRPFD1Y,TC00000004
185029,E24W06W0290,202406,WSSH03003WRPFD1Y,TC00000002
185030,E24F07W0002,202407,FSSV062U2RBRTV1Y,ISS6605004
185031,D24F08W0002,202408,FVAA001AMFBMDY1Y,ISS6616002


In [10]:
# 특정 컬럼명 변경
design_code.rename(columns={'PREQ_NO': '생산의뢰번호',
                            'STD_ITM_CODE':'표준품목코드',
                            'DSIGN_CON_CODE':'설계조건코드'}, inplace=True)

# 결과 확인
design_code

,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,D20N05W0002,202101,NSSF006U1DCBSY1Y,IS21464001
1,D20F07W0580,202101,FSMF00800RBRTY1Y,IC00000001
2,D20F08W0322,202101,FTLE00200FBBSY1Y,IS25469001
3,D20F06W0539,202101,FSMA06100RBRTY1Y,IS21464002
4,D20F07W0480,202101,FTLE00200FBMDY1Y,IS25469010
...,...,...,...,...
185028,E24W06W0287,202406,WSSH03003WRPFD1Y,TC00000004
185029,E24W06W0290,202406,WSSH03003WRPFD1Y,TC00000002
185030,E24F07W0002,202407,FSSV062U2RBRTV1Y,ISS6605004
185031,D24F08W0002,202408,FVAA001AMFBMDY1Y,ISS6616002


In [11]:
# query_data와 design_code 데이터프레임을 '생산의뢰번호'기준으로 inner merge
query_data = pd.merge(query_data, design_code, how='inner', on='생산의뢰번호')

# 결과 확인
query_data

,LOT_NO,사내강종,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,재료코드,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [12]:
# 컬럼별 null 데이터 갯수 확인
column_null_counts = query_data.isnull().sum(axis=0)

# 결과 확인
print(column_null_counts)

LOT_NO          0
사내강종            0
사내강종명           0
품종              0
주문형상            0
열처리        889385
표면              0
설계외경            0
설계폭             0
최초투입형상          0
HEAT_NO         0
재료코드        20140
소재_외경           0
소재_폭            0
공정순위            0
공정코드            0
공정명             0
투입정보_형상    181242
투입정보_외경     18140
투입정보_폭      18190
제소구분            0
생산의뢰번호          0
수주년월            0
표준품목코드          0
설계조건코드          0
dtype: int64


In [13]:
# 특정컬럼의 고유값 확인
unique_values = query_data['열처리'].unique()

# 결과 확인
unique_values

array(['SA', 'SL', 'FA', None, 'NQTT', 'QTT', 'QT', 'LA', 'NSQT', 'A',
       'SLA', 'N', 'QTS', 'AG', 'NSA', 'NT', 'SRA', 'DS', 'NH', 'NQT',
       'SLBA', 'NQS'], dtype=object)

In [14]:
# 특정컬럼의 null 데이터에 특정값 입력
query_data['열처리'] = query_data['열처리'].fillna('NH')

# 결과 확인
query_data

,LOT_NO,사내강종,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,재료코드,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [15]:
# 새로운 파생변수 생성
query_data['생산의뢰번호_대분류'] = query_data['생산의뢰번호'].str.slice(0, 1)

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('생산의뢰번호')
query_data.insert(position, '생산의뢰번호_대분류', query_data.pop('생산의뢰번호_대분류'))

# 결과 확인
query_data

,LOT_NO,사내강종,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,재료코드,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [16]:
# 특정컬럼의 고유값 확인
unique_values = query_data['생산의뢰번호_대분류'].unique()

# 결과 확인
unique_values

array(['E', 'D', 'G', 'L', 'S', 'R', 'H', 'P', 'O', 'W', 'J'],
      dtype=object)

In [17]:
# 피벗테이블 생성
pivot_table = query_data.pivot_table(values='LOT_NO',
                                     index=['생산의뢰번호_대분류','생산의뢰번호'],
                                     aggfunc="count").reset_index()

# 특정컬럼을 기준으로 데이터 내림차순 정렬
pivot_table.reset_index(drop=True, inplace=True)

# 결과 확인
pivot_table

,생산의뢰번호_대분류,생산의뢰번호,LOT_NO
0,D,D20F02W0123,21
1,D,D20F12W0420,14
2,D,D21B09W1230,19
3,D,D21B11W0514,54
4,D,D21F02W0399,28
...,...,...,...
62671,W,W23S12W0018,32
62672,W,W24F03W0004,18
62673,W,W24F03W0005,80
62674,W,W24F03W0010,18


In [18]:
# 특정컬럼의 데이터가 특정고유값을 가지는 row데이터 추출
query_data = query_data[query_data['생산의뢰번호_대분류'].isin(['D','E','G'])] # D(내수품), E(수출품), G(강관원재)

# 결과 확인
query_data

,LOT_NO,사내강종,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,재료코드,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [19]:
# 특정컬럼의 고유값 확인
unique_values = query_data['최초투입형상'].unique()

# 결과 확인
unique_values

array(['BM', 'RB', 'BT', 'IG', 'LT', 'WR', 'FB', 'SB'], dtype=object)

In [20]:
# 특정컬럼의 데이터가 특정고유값을 가지는 row데이터 추출
query_data = query_data[query_data['최초투입형상'].isin(['IG','BT','BM'])]

# 결과 확인
query_data

,LOT_NO,사내강종,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,재료코드,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,P250,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,7730,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [21]:
# 새로운 파생변수 생성
query_data['HEAT_NO_구분'] = query_data['HEAT_NO'].str.slice(0, 1)

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('재료코드')
query_data.insert(position, 'HEAT_NO_구분', query_data.pop('HEAT_NO_구분'))

# 결과 확인
query_data

C:\Users\seah\AppData\Local\Temp\ipykernel_21124\202804197.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_data['HEAT_NO_구분'] = query_data['HEAT_NO'].str.slice(0, 1)


,LOT_NO,사내강종,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [22]:
# 특정컬럼의 고유값 확인
unique_values = query_data['HEAT_NO_구분'].unique()

# 결과 확인
unique_values

array(['N', 'S', 'R', 'H', 'V', 'K', 'E'], dtype=object)

In [23]:
# 특정컬럼의 데이터가 특정고유값을 제외한 row데이터 추출
query_data = query_data[~(query_data['HEAT_NO_구분'] == 'E')] # E(구입재_HEAT_NO)

# 결과 확인
query_data

,LOT_NO,사내강종,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [24]:
# 새로운 파생변수 생성
query_data['재료코드_구분'] = query_data['재료코드'].str.slice(0, 1)

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('소재_외경')
query_data.insert(position, '재료코드_구분', query_data.pop('재료코드_구분'))

# 결과 확인
query_data

C:\Users\seah\AppData\Local\Temp\ipykernel_21124\3260308389.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_data['재료코드_구분'] = query_data['재료코드'].str.slice(0, 1)


,LOT_NO,사내강종,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [25]:
# 특정컬럼의 고유값 확인
unique_values = query_data['재료코드_구분'].unique()

# 결과 확인
unique_values

array(['P', '0', '7', 'D', 'J', 'I', 'T', 'O', '6', 'Y', 'E', 'R', 'U',
       'M', None, 'N', 'Z'], dtype=object)

In [26]:
# 특정컬럼의 데이터가 null이 포함된 row데이터 추출
null_data = query_data.loc[query_data['재료코드_구분'].isnull()]

# 특정컬럼 데이터가 null이 포함된 LOT_NO정보 추출 (대부분 전환인수나, 스크랩 재생된 LOT로 보임)
unique_values = null_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B320030900', 'B320031300', 'B320030800', 'B320031500',
       'B320031000', 'B320031200', 'B320031100', 'B320031400',
       'B320555800', 'B320555900', 'B330185900', 'B330185901',
       'B330185902', 'B330186000', 'B330186001', 'B330186002',
       'B380251300', 'B380251301', 'B380251302', 'B3A0254400',
       'B3A0254600', 'B3A0254500', 'B3A0254700', 'B380251200',
       'B380251401', 'B380251201', 'B380251202', 'B3A0254200',
       'B380251000', 'B380251001', 'B380251002', 'B380250900',
       'B380250901', 'B380250902', 'B380251601', 'B380252101',
       'B380252100', 'B380251500', 'B380251400', 'B380251600',
       'B380251100', 'B380250800', 'B380250803', 'B380251101',
       'B380250802', 'B380251102', 'B380250801', 'B3A0254300',
       'B3A0254301', 'B3A0254302', 'B380252700', 'B380252701',
       'B380252400', 'B380252200', 'B380251800', 'B380251802',
       'B380252401', 'B380252402', 'B380252201', 'B380251900',
       'B380251901', 'B380252800', 'B380252501', 'B3802

In [27]:
# unique_values의 갯수 확인
# 길이 구하기
length = len(unique_values)

# 결과 확인
print(length)

69


In [28]:
# unique_values에 해당하는 row데이터 삭제
query_data = query_data[~np.isin(query_data['LOT_NO'], unique_values)]

# 결과 확인
query_data

,LOT_NO,사내강종,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [29]:
# 새로운 파생변수 생성
query_data['대강종코드'] = query_data['사내강종'].str.slice(0, 1)

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('사내강종명')
query_data.insert(position, '대강종코드', query_data.pop('대강종코드'))

# 결과 확인
query_data

C:\Users\seah\AppData\Local\Temp\ipykernel_21124\2489352328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_data['대강종코드'] = query_data['사내강종'].str.slice(0, 1)


,LOT_NO,사내강종,대강종코드,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,T,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,T,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,T,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,T,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,T,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,S,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,S,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,S,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,S,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [30]:
# 특정컬럼의 고유값 확인
unique_values = query_data['대강종코드'].unique()

# 결과 확인
unique_values

array(['T', 'S', 'B', 'A', 'V', 'C', 'M', 'K'], dtype=object)

In [31]:
# 조건에 맞는 row데이터 선택하여 새로운 파생변수에 대체할 문자로 채워넣기
query_data.loc[query_data['대강종코드'].isin(['A','B','C','K','M']), '대강종명'] = '탄소합금강'
query_data.loc[query_data['대강종코드'] == 'S', '대강종명'] = '스테인리스강'
query_data.loc[query_data['대강종코드'] == 'T', '대강종명'] = '공구강'
query_data.loc[query_data['대강종코드'] == 'V', '대강종명'] = '특수합금'

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('대강종코드')
query_data.insert(position, '대강종명', query_data.pop('대강종명'))

# 결과 확인
query_data

C:\Users\seah\AppData\Local\Temp\ipykernel_21124\1281828399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_data.loc[query_data['대강종코드'].isin(['A','B','C','K','M']), '대강종명'] = '탄소합금강'


,LOT_NO,사내강종,대강종명,대강종코드,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [32]:
# 특정컬럼의 고유값 확인
unique_values = query_data['대강종명'].unique()

# 결과 확인
unique_values

array(['공구강', '스테인리스강', '탄소합금강', '특수합금'], dtype=object)

In [33]:
# 새로운 파생변수 생성
query_data['대중강종코드'] = query_data['사내강종'].str.slice(0, 2)

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('사내강종명')
query_data.insert(position, '대중강종코드', query_data.pop('대중강종코드'))

# 결과 확인
query_data

C:\Users\seah\AppData\Local\Temp\ipykernel_21124\3318265437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_data['대중강종코드'] = query_data['사내강종'].str.slice(0, 2)


,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [34]:
# 새로운 파생변수 생성
query_data['대중소강종코드'] = query_data['사내강종'].str.slice(0, 3)

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('사내강종명')
query_data.insert(position, '대중소강종코드', query_data.pop('대중소강종코드'))

# 결과 확인
query_data

C:\Users\seah\AppData\Local\Temp\ipykernel_21124\3816543814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_data['대중소강종코드'] = query_data['사내강종'].str.slice(0, 3)


,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [35]:
# 특정컬럼의 고유값 확인
unique_values = query_data['품종'].unique()

# 결과 확인
unique_values

array(['S', 'B', 'F', 'W', 'Z'], dtype=object)

In [36]:
# 조건에 맞는 row데이터 선택하여 새로운 파생변수에 대체할 문자로 채워넣기
query_data.loc[query_data['품종'] == 'S', '품종명'] = '소형재'
query_data.loc[query_data['품종'] == 'B', '품종명'] = '대형재'
query_data.loc[query_data['품종'] == 'F', '품종명'] = '일반단조'
query_data.loc[query_data['품종'] == 'W', '품종명'] = '선재'
query_data.loc[query_data['품종'] == 'Z', '품종명'] = '반제품'

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('주문형상')
query_data.insert(position, '품종명', query_data.pop('품종명'))

# 결과 확인
query_data

C:\Users\seah\AppData\Local\Temp\ipykernel_21124\2509618610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_data.loc[query_data['품종'] == 'S', '품종명'] = '소형재'


,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [37]:
# 특정컬럼의 고유값 확인
unique_values = query_data['품종명'].unique()

# 결과 확인
unique_values

array(['소형재', '대형재', '일반단조', '선재', '반제품'], dtype=object)

In [38]:
# 새로운 파생변수 생성
query_data['공정구분'] = query_data['공정코드'].str.slice(0, 1)

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('공정명')
query_data.insert(position, '공정구분', query_data.pop('공정구분'))

# 결과 확인
query_data

C:\Users\seah\AppData\Local\Temp\ipykernel_21124\872483265.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_data['공정구분'] = query_data['공정코드'].str.slice(0, 1)


,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [39]:
# 특정컬럼의 고유값 확인
unique_values = query_data['공정구분'].unique()

# 결과 확인
unique_values

array(['6', 'S', '7', '8', 'B', 'A', 'L', 'F', '3', '2', '9', 'E', 'C',
       'R', '5', 'G', 'H'], dtype=object)

In [40]:
# unique_values의 갯수 확인
# 길이 구하기
length = len(unique_values)

# 결과 확인
print(length)

17


In [41]:
# 조건에 맞는 row데이터 선택하여 새로운 파생변수에 대체할 문자로 채워넣기
query_data.loc[query_data['공정구분'] == '6', '공장구분'] = '1단조'
query_data.loc[query_data['공정구분'] == 'F', '공장구분'] = '2단조'
query_data.loc[query_data['공정구분'] == 'R', '공장구분'] = 'RFM'
query_data.loc[query_data['공정구분'] == 'A', '공장구분'] = '가공공장'
query_data.loc[query_data['공정구분'] == 'L', '공장구분'] = '대형정정'
query_data.loc[query_data['공정구분'] == '8', '공장구분'] = '소형압연'
query_data.loc[query_data['공정구분'] == 'B', '공장구분'] = '소형정정'
query_data.loc[query_data['공정구분'].isin(['2', '5']), '공장구분'] = '2제강'
query_data.loc[query_data['공정구분'].isin(['G', 'H']), '공장구분'] = '3제강'
query_data.loc[query_data['공정구분'].isin(['7', 'S']), '공장구분'] = '대형압연'
query_data.loc[query_data['공정구분'].isin(['9', 'E']), '공장구분'] = '산세공장'
query_data.loc[query_data['공정구분'].isin(['3', 'C']), '공장구분'] = '특수제강'

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('공정명')
query_data.insert(position, '공장구분', query_data.pop('공장구분'))

# 결과 확인
query_data

C:\Users\seah\AppData\Local\Temp\ipykernel_21124\3003794921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_data.loc[query_data['공정구분'] == '6', '공장구분'] = '1단조'


,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [42]:
# 특정컬럼의 고유값 확인
unique_values = query_data['공장구분'].unique()

# 결과 확인
unique_values

array(['1단조', '대형압연', '소형압연', '소형정정', '가공공장', '대형정정', '2단조', '특수제강',
       '2제강', '산세공장', 'RFM', '3제강'], dtype=object)

In [43]:
# 특정컬럼의 데이터가 Null인 row 데이터 추출
null_data = query_data.loc[query_data['투입정보_형상'].isnull()]

# 결과 확인
null_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
8,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,70,8W31,8,소형압연,선재-선재압연검사,None,0.0,0.0,2,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
164,B2A0161500,TKA002,공구강,T,TK,TKA,STD11S0,S,소형재,RB,SA,PM,36.0,36.0,IG,N28055,N,D013,D,442.0,290.0,100,8W31,8,소형압연,선재-선재압연검사,None,0.0,0.0,2,E,E22S10W0085,202210,STKA002U2RBPMY1Y,IC00000006
326,B2B0193300,TKA003,공구강,T,TK,TKA,STD11S1,S,소형재,RB,SA,PM,38.0,38.0,IG,N28326,N,D013,D,442.0,290.0,100,8W31,8,소형압연,선재-선재압연검사,None,0.0,0.0,2,E,E22S10W0079,202210,STKA00301RBPMY1Y,ISS1315004
344,B2B0204100,TKB085,공구강,T,TK,TKB,STD61S0,S,소형재,RB,SA,PM,61.0,61.0,IG,N28327,N,D013,D,442.0,290.0,90,8W31,8,소형압연,선재-선재압연검사,None,0.0,0.0,2,E,E22S11W0200,202211,STKB085U4RBPMY1Y,IC00000007
615,B250802600,SSU001,스테인리스강,S,SS,SSU,F60S1,S,소형재,RB,SL,PM,30.0,30.0,BT,S50390,S,6830,6,160.0,160.0,60,8W31,8,소형압연,선재-선재압연검사,None,0.0,0.0,2,D,D22S06W0937,202206,SSSU001U1RBPMY1Y,IS21464006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2813943,B440597403,SMM015,스테인리스강,S,SM,SMM,STS430FS6,W,선재,WR,FA,PK,11.5,11.5,BT,S57242,S,7730,7,170.0,170.0,70,9W41,9,산세공장,코일-열처리검사,None,0.0,0.0,2,E,E24W05W0368,202405,WSMM01500WRPKD1Y,TSS1113001
2813955,B450373101,SSA151,스테인리스강,S,SS,SSA,STS304MS2,W,선재,WR,SL,PK,22.0,22.0,BT,S57618,S,7730,7,170.0,170.0,60,9W41,9,산세공장,코일-열처리검사,None,0.0,0.0,2,D,D24W05W1108,202405,WSSA151U5WRPKQ1Y,TS21583003
2814067,B440634701,SSM053,스테인리스강,S,SS,SSM,316LDS2,W,선재,WR,SL,PK,32.0,32.0,BT,S57287,S,7730,7,170.0,170.0,60,9W41,9,산세공장,코일-열처리검사,None,0.0,0.0,2,D,D24W05W1343,202405,WSSM053U1WRPKD1Y,TS20229007
2814091,B450179045,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,80,9W41,9,산세공장,코일-열처리검사,None,NaN,NaN,2,D,D24S05W0273,202405,SSUB007U2RBPMF1Y,TSS0531001


In [44]:
# 특정컬럼의 고유값 확인
unique_values = null_data['공정명'].unique()

# 결과 확인
unique_values

array(['선재-선재압연검사', '코일-열처리검사', '선재-DST', '선재-수입검사', '소형정정-평강교정기',
       '분괴-분괴압연검사', '특가-대형절단기(1080B/S)', '선재-연속가열로', '선재-압연공통',
       '분괴-Bump교정기', '가공-배치어닐링로 1호', '가공-백피교정기10', '가공-입회검사', '가공-열처리검사',
       '특가-대형절단기(1300B/S) 1호', '3제강-특제-조괴검사', '코일-입고검사', '가공-대형절단기1080',
       '분괴-SBM-압연', '가공-대형절단기430 1호', '분괴-Bump교정기-1500톤', '분괴-쇼트브라스트기',
       '가공-중간검사', '2단조-9000톤 프레스', '가공-입고', '특가-입회검사', '코일-연속고용화로',
       '가공-백피교정기160', '가공-ET/UT35', '분괴-환봉각재 압연기', '소형정정-대형절단기700 1호',
       '가공-흑피교정기100', '가공-흑피교정기80', '가공-소형절단400 1호', '가공-ET/UT320',
       '코일-수동연마', '선재-연마 1호', '가공-배치템퍼링로 1호', '코일-쇼트브라스트기', '코일-중간검사',
       '2단조-절단 1호-100톤(Flame)', '특가-황삭외주'], dtype=object)

In [45]:
# 특정공정이 Skip된 row 데이터 추출
skip_data = null_data.loc[null_data['공정명'] == '가공-ET/UT35']

# 특정공정이 Skip된 row 데이터의 LOT_NO 고유값 확인
unique_values = skip_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B390114420', 'B390114410', 'B390114414', 'B390114415',
       'B390114416', 'B390114804', 'B390114844', 'B390114845',
       'B390114846', 'B390114847', 'B390114848', 'B390114849',
       'B390114803', 'B390114843', 'B390114835', 'B390114836',
       'B390114837', 'B390114838', 'B390114839', 'B390114841',
       'B390114842', 'B390114822', 'B390114823', 'B390114825',
       'B390114826', 'B390114829', 'B390114830', 'B390114831',
       'B390114832', 'B390114833', 'B380148952', 'B380148953'],
      dtype=object)

In [46]:
# 특정공정이 Skip된 row 데이터 삭제
query_data = query_data.dropna(subset=['투입정보_형상'])

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


### 3. 2차 데이터 전처리 ###

In [47]:
# 컬럼별 null 데이터 갯수 확인
column_null_counts = query_data.isnull().sum(axis=0)

# 결과 확인
print(column_null_counts)

LOT_NO         0
사내강종           0
대강종명           0
대강종코드          0
대중강종코드         0
대중소강종코드        0
사내강종명          0
품종             0
품종명            0
주문형상           0
열처리            0
표면             0
설계외경           0
설계폭            0
최초투입형상         0
HEAT_NO        0
HEAT_NO_구분     0
재료코드           0
재료코드_구분        0
소재_외경          0
소재_폭           0
공정순위           0
공정코드           0
공정구분           0
공장구분           0
공정명            0
투입정보_형상        0
투입정보_외경        0
투입정보_폭        47
제소구분           0
생산의뢰번호_대분류     0
생산의뢰번호         0
수주년월           0
표준품목코드         0
설계조건코드         0
dtype: int64


In [48]:
# 숫자데이터를 포함하는 컬럼들의 기초통계정보 확인
query_data.describe()

,설계외경,설계폭,소재_외경,소재_폭,공정순위,투입정보_외경,투입정보_폭,수주년월
count,2.568865e+06,2.568865e+06,2.568865e+06,2.568865e+06,2.568865e+06,2.568865e+06,2.568818e+06,2.568865e+06
mean,6.177264e+01,9.435807e+01,3.467781e+02,2.989931e+02,6.732948e+01,1.245318e+02,1.150674e+02,2.023255e+05
std,9.532675e+01,1.592856e+02,2.841793e+02,1.978055e+02,5.021426e+01,1.448540e+02,1.797885e+02,5.067511e+01
min,4.500000e+00,4.500000e+00,9.500000e+01,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,2.021050e+05
25%,1.000000e+01,1.000000e+01,1.700000e+02,1.700000e+02,3.000000e+01,1.800000e+01,0.000000e+00,2.023030e+05
50%,2.300000e+01,2.500000e+01,1.700000e+02,1.700000e+02,6.000000e+01,1.000000e+02,0.000000e+00,2.023080e+05
75%,6.500000e+01,1.120000e+02,3.900000e+02,4.700000e+02,9.000000e+01,1.700000e+02,1.700000e+02,2.024010e+05
max,1.350000e+03,2.200000e+03,1.770000e+03,1.530000e+03,6.900000e+02,1.770000e+03,2.510000e+03,2.024060e+05


In [49]:
# 특정조건을 만족하는 row데이터 추출
filtered_data = query_data[(query_data['최초투입형상'] == 'BM') &
                           (query_data['재료코드_구분'] == 'P') &
                           (query_data['소재_외경'] != 390)]

# 특정조건을 만족하는 row데이터의 특정컬럼의 고유값 추출
unique_values = filtered_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B360320000'], dtype=object)

In [50]:
# unique_values에 해당하는 row데이터 삭제
query_data = query_data[~np.isin(query_data['LOT_NO'], unique_values)]

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [51]:
# 특정조건을 만족하는 row데이터 추출
filtered_data = query_data[(query_data['투입정보_형상'] == 'BM') &
                           (query_data['투입정보_외경'] != 390)]

# 특정조건을 만족하는 row데이터의 특정컬럼의 고유값 추출
unique_values = filtered_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B350455100'], dtype=object)

In [52]:
# unique_values에 해당하는 row데이터 삭제
query_data = query_data[~np.isin(query_data['LOT_NO'], unique_values)]

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [53]:
# 특정조건을 만족하는 row데이터 추출
filtered_data = query_data[(query_data['공정코드'] == '5V60')] # 2제강_외판빌렛인수공정

# 특정조건을 만족하는 row데이터의 특정컬럼의 고유값 추출
unique_values = filtered_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B270532954', 'B260156401', 'B270453501', ..., 'B440123811',
       'B440123829', 'B3B0442137'], dtype=object)

In [54]:
# 특정조건을 만족하는 row데이터 추출
query_data = query_data[~(query_data['공정코드'] == '5V60')] # 2제강_외판빌렛인수공정

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [55]:
# 특정조건을 만족하는 row데이터 추출
filtered_data = query_data[query_data['공정코드'] == 'FV70']  #2단조_스크랩재생인수공정

# 특정조건을 만족하는 row데이터의 특정컬럼의 고유값 추출
unique_values = filtered_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B320427700', 'B320427800', 'B320427801', 'B320427900'],
      dtype=object)

In [56]:
# unique_values에 해당하는 row데이터 삭제
query_data = query_data[~np.isin(query_data['LOT_NO'], unique_values)]

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [57]:
# 특정조건을 만족하는 row데이터 추출
filtered_data = query_data[query_data['공정코드'] == '8V30'] # 선재_합병투입

# 특정조건을 만족하는 row데이터의 특정컬럼의 고유값 추출
unique_values = filtered_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B340223000', 'B340261400', 'B340505900', ..., 'B450206506',
       'B450206507', 'B450206508'], dtype=object)

In [58]:
# unique_values의 갯수 확인
# 길이 구하기
length = len(unique_values)

# 결과 확인
print(length)

1380


In [59]:
# unique_values에 해당하는 row데이터 삭제
query_data = query_data[~np.isin(query_data['LOT_NO'], unique_values)]

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [60]:
# 특정조건을 만족하는 row데이터 추출
filtered_data = query_data[query_data['공정코드'] == '8P01'] # 선재_압연 임가공

# 특정조건을 만족하는 row데이터의 특정컬럼의 고유값 추출
unique_values = filtered_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B2C0211700', 'B2B0561500', 'B2C0207900', ..., 'B450205706',
       'B450205707', 'B450205708'], dtype=object)

In [61]:
# unique_values의 갯수 확인
# 길이 구하기
length = len(unique_values)

# 결과 확인
print(length)

11336


In [62]:
# unique_values에 해당하는 row데이터 삭제
query_data = query_data[~np.isin(query_data['LOT_NO'], unique_values)]

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [63]:
# 특정조건을 만족하는 row데이터 추출
filtered_data = query_data.loc[query_data['공정명'].str.contains('입회')]

# 특정조건을 만족하는 row데이터의 특정컬럼의 고유값 추출
unique_values = filtered_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B2B0043000', 'B2B0043001', 'B2B0043002', 'B2B0043003',
       'B2B0043004', 'B2B0043005', 'B2B0043006', 'B2B0043007',
       'B2B0043008', 'B270457604', 'B270457610', 'B2A0372003',
       'B2A0372004', 'B2A0372006', 'B2A0372007', 'B2A0372001',
       'B2A0372009', 'B2A0372010', 'B2A0372011', 'B2A0372000',
       'B2B0331201', 'B2B0331202', 'B2B0331206', 'B2B0331207',
       'B220683700', 'B270457807', 'B270457811', 'B2B0331203',
       'B2B0331204', 'B2C0193606', 'B2C0193502', 'B2B0331200',
       'B2C0193600', 'B2C0193503', 'B2C0193504', 'B260437414',
       'B270430201', 'B230476000', 'B230476003', 'B250033905',
       'B250033907', 'B250033915', 'B250033900', 'B2C0193601',
       'B2C0193603', 'B2C0193604', 'B2C0193505', 'B260437422',
       'B2B0192700', 'B2B0192701', 'B2B0265601', 'B2B0426501',
       'B2B0426502', 'B2B0426505', 'B2B0426506', 'B2B0426509',
       'B2B0426511', 'B2C0193605', 'B2C0193607', 'B2B0192702',
       'B2B0265600', 'B2C0193501', 'B2C0193507', 'B2B04

In [64]:
# unique_values의 갯수 확인
# 길이 구하기
length = len(unique_values)

# 결과 확인
print(length)

567


In [65]:
# 특정컬럼의 데이터가 특정문구를 포함한 index 데이터를 제외
query_data = query_data.loc[~(query_data['공정명'].str.contains('입회'))]

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [66]:
# 컬럼별 null 데이터 갯수 확인
column_null_counts = query_data.isnull().sum(axis=0)

# 결과 확인
print(column_null_counts)

LOT_NO         0
사내강종           0
대강종명           0
대강종코드          0
대중강종코드         0
대중소강종코드        0
사내강종명          0
품종             0
품종명            0
주문형상           0
열처리            0
표면             0
설계외경           0
설계폭            0
최초투입형상         0
HEAT_NO        0
HEAT_NO_구분     0
재료코드           0
재료코드_구분        0
소재_외경          0
소재_폭           0
공정순위           0
공정코드           0
공정구분           0
공장구분           0
공정명            0
투입정보_형상        0
투입정보_외경        0
투입정보_폭        47
제소구분           0
생산의뢰번호_대분류     0
생산의뢰번호         0
수주년월           0
표준품목코드         0
설계조건코드         0
dtype: int64


In [67]:
# 숫자데이터를 포함하는 컬럼들의 기초통계정보 확인
query_data.describe()

,설계외경,설계폭,소재_외경,소재_폭,공정순위,투입정보_외경,투입정보_폭,수주년월
count,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06,2.482941e+06,2.482988e+06
mean,6.224799e+01,9.595970e+01,3.523222e+02,3.030314e+02,6.725615e+01,1.258565e+02,1.172675e+02,2.023253e+05
std,9.675968e+01,1.616792e+02,2.869417e+02,1.996262e+02,5.050755e+01,1.466069e+02,1.818361e+02,5.070927e+01
min,4.500000e+00,4.500000e+00,9.500000e+01,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,2.021050e+05
25%,1.000000e+01,1.000000e+01,1.700000e+02,1.700000e+02,3.000000e+01,1.700000e+01,0.000000e+00,2.023030e+05
50%,2.300000e+01,2.324000e+01,1.700000e+02,1.700000e+02,6.000000e+01,1.050000e+02,0.000000e+00,2.023080e+05
75%,6.700000e+01,1.240000e+02,4.300000e+02,4.700000e+02,9.000000e+01,1.700000e+02,1.700000e+02,2.024010e+05
max,1.350000e+03,2.200000e+03,1.770000e+03,1.530000e+03,6.900000e+02,1.770000e+03,2.510000e+03,2.024060e+05


In [68]:
# 특정컬럼의 데이터가 특정값(이상치)를 포함한 index의 데이터 추출
filtered_data = query_data.loc[query_data['투입정보_외경'] == 1]

# 특정컬럼의 컬럼의 데이터가 특정값(이상치)를 포함한 LOT_NO 데이터 추출
unique_values = filtered_data['LOT_NO'].unique()
unique_values

array(['B320560008', 'B320560013'], dtype=object)

In [69]:
# 특정컬럼의 컬럼의 데이터가 특정값(이상치)를 포함한 row데이터를 특정값(정상값) 데이터로 변환
mask = query_data['투입정보_외경'] == 1
query_data.loc[mask, '투입정보_외경'] = 55

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [70]:
# 숫자데이터를 포함하는 컬럼들의 기초통계정보 확인
query_data.describe()

,설계외경,설계폭,소재_외경,소재_폭,공정순위,투입정보_외경,투입정보_폭,수주년월
count,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06,2.482941e+06,2.482988e+06
mean,6.224799e+01,9.595970e+01,3.523222e+02,3.030314e+02,6.725615e+01,1.258566e+02,1.172675e+02,2.023253e+05
std,9.675968e+01,1.616792e+02,2.869417e+02,1.996262e+02,5.050755e+01,1.466069e+02,1.818361e+02,5.070927e+01
min,4.500000e+00,4.500000e+00,9.500000e+01,0.000000e+00,1.000000e+00,4.500000e+00,0.000000e+00,2.021050e+05
25%,1.000000e+01,1.000000e+01,1.700000e+02,1.700000e+02,3.000000e+01,1.700000e+01,0.000000e+00,2.023030e+05
50%,2.300000e+01,2.324000e+01,1.700000e+02,1.700000e+02,6.000000e+01,1.050000e+02,0.000000e+00,2.023080e+05
75%,6.700000e+01,1.240000e+02,4.300000e+02,4.700000e+02,9.000000e+01,1.700000e+02,1.700000e+02,2.024010e+05
max,1.350000e+03,2.200000e+03,1.770000e+03,1.530000e+03,6.900000e+02,1.770000e+03,2.510000e+03,2.024060e+05


In [71]:
# 특정컬럼의 컬럼의 데이터가 Null값을 포함한 row 데이터 추출
null_data = query_data.loc[query_data['투입정보_폭'].isnull()]

# 특정컬럼의 컬럼의 데이터가 Null값을 포함한 LOT_NO 데이터 추출
unique_values = null_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B2C0554101', 'B2C0554100', 'B330292900', 'B330292902',
       'B330292903', 'B330292901', 'B330289800'], dtype=object)

In [72]:
# unique_values의 갯수 확인
# 길이 구하기
length = len(unique_values)

# 결과 확인
print(length)

7


In [73]:
# 특정컬럼의 고유값 확인
unique_values = null_data['투입정보_형상'].unique()

# 결과 확인
unique_values

array(['RB'], dtype=object)

In [74]:
# 특정컬럼의 데이터가 Null값을 가지는 row 데이터를 다른컬럼의 데이터로 대체하기
mask = query_data['투입정보_폭'].isnull()
query_data.loc[mask, '투입정보_폭'] = query_data['투입정보_외경']

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,0.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [75]:
# 특정컬럼의 데이터가 zero값을 포함한 row 데이터 추출
zero_data = query_data.loc[query_data['투입정보_폭'] == 0]

# 특정컬럼의 데이터가 zero값을 포함한 LOT_NO 데이터 추출
unique_values = zero_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B260147200', 'B2C0199200', 'B0C0740100', ..., 'B450040201',
       'B450179045', 'B450179028'], dtype=object)

In [76]:
# unique_values의 갯수 확인
# 길이 구하기
length = len(unique_values)

# 결과 확인
print(length)

250378


In [77]:
# 특정컬럼의 고유값 확인
unique_values = null_data['투입정보_형상'].unique()

# 결과 확인
unique_values

array(['RB'], dtype=object)

In [78]:
# 특정컬럼의 데이터가 zero값을 가지는 row 데이터를 다른컬럼의 데이터로 대체하기
mask = query_data['투입정보_폭'] == 0
query_data.loc[mask, '투입정보_폭'] = query_data['투입정보_외경']

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,2,6105,6,1단조,단조-1800톤 프레스,BM,390.0,470.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
1,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,4,6W31,6,1단조,단조-단조검사,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
2,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,10,S105,S,대형압연,분괴-SBM-압연,SL,260.0,410.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
3,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,20,SW31,S,대형압연,분괴-SBM-압연검사,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
4,B260147200,TTA002,공구강,T,TT,TTA,STC3S1,S,소형재,RB,SA,PM,80.0,80.0,BM,N27187,N,P250,P,390.0,470.0,30,7402,7,대형압연,분괴-Bump교정기-1500톤,LT,160.0,160.0,1,E,E22S04W0360,202204,STTA00200RBPMY1Y,BC00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814111,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,10,B201,B,소형정정,소형정정-연속어닐링로 1호,BT,170.0,170.0,1,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814112,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,120,AB03,A,가공공장,가공-소형절단400 1호,RB,16.0,16.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814113,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,130,AW93,A,가공공장,가공-입고검사C,RB,16.0,16.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001
2814114,B450179028,SUB007,스테인리스강,S,SU,SUB,STR11S1,S,소형재,RB,FA,PM,16.0,16.0,BT,S57517,S,7730,7,170.0,170.0,140,AZ07,A,가공공장,가공-외주포장,RB,16.0,16.0,2,D,D24S05W0269,202405,SSUB007U1RBPMF1Y,TS20274001


In [79]:
# 컬럼별 null 데이터 갯수 확인
column_null_counts = query_data.isnull().sum(axis=0)

# 결과 확인
print(column_null_counts)

LOT_NO        0
사내강종          0
대강종명          0
대강종코드         0
대중강종코드        0
대중소강종코드       0
사내강종명         0
품종            0
품종명           0
주문형상          0
열처리           0
표면            0
설계외경          0
설계폭           0
최초투입형상        0
HEAT_NO       0
HEAT_NO_구분    0
재료코드          0
재료코드_구분       0
소재_외경         0
소재_폭          0
공정순위          0
공정코드          0
공정구분          0
공장구분          0
공정명           0
투입정보_형상       0
투입정보_외경       0
투입정보_폭        0
제소구분          0
생산의뢰번호_대분류    0
생산의뢰번호        0
수주년월          0
표준품목코드        0
설계조건코드        0
dtype: int64


In [80]:
# 숫자데이터를 포함하는 컬럼들의 기초통계정보 확인
query_data.describe()

,설계외경,설계폭,소재_외경,소재_폭,공정순위,투입정보_외경,투입정보_폭,수주년월
count,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06,2.482988e+06
mean,6.224799e+01,9.595970e+01,3.523222e+02,3.030314e+02,6.725615e+01,1.258566e+02,1.541444e+02,2.023253e+05
std,9.675968e+01,1.616792e+02,2.869417e+02,1.996262e+02,5.050755e+01,1.466069e+02,1.761680e+02,5.070927e+01
min,4.500000e+00,4.500000e+00,9.500000e+01,0.000000e+00,1.000000e+00,4.500000e+00,4.500000e+00,2.021050e+05
25%,1.000000e+01,1.000000e+01,1.700000e+02,1.700000e+02,3.000000e+01,1.700000e+01,1.741000e+01,2.023030e+05
50%,2.300000e+01,2.324000e+01,1.700000e+02,1.700000e+02,6.000000e+01,1.050000e+02,1.550000e+02,2.023080e+05
75%,6.700000e+01,1.240000e+02,4.300000e+02,4.700000e+02,9.000000e+01,1.700000e+02,1.700000e+02,2.024010e+05
max,1.350000e+03,2.200000e+03,1.770000e+03,1.530000e+03,6.900000e+02,1.770000e+03,2.510000e+03,2.024060e+05


In [81]:
# 특정컬럼의 데이터가 zero값을 포함한 row 데이터 추출
zero_data = query_data.loc[query_data['소재_폭'] == 0]

# 특정컬럼의 데이터가 zero값을 포함한 LOT_NO 데이터 추출
unique_values = zero_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['B370329600', 'B370329602', 'B370329603', 'B370329606',
       'B370329601', 'B3C0401700', 'B370329700'], dtype=object)

### 4. 데이터 Pivoting ###

In [82]:
# 특정컬럼을 기준으로 데이터 내림차순 정렬
query_data = query_data.sort_values(by=['LOT_NO', '공정순위'], ascending=True)
query_data.reset_index(drop=True, inplace=True)

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,8601,8,소형압연,선재-연속가열로,BT,170.0,170.0,1,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,8100,8,소형압연,선재-압연공통,BT,170.0,170.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,8W31,8,소형압연,선재-선재압연검사,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,B201,B,소형정정,소형정정-연속어닐링로 1호,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,BW41,B,소형정정,소형정정-열처리검사,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,5Y01,5,2제강,2제강-연주-입고,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,5W91,5,2제강,2제강-연주-입고검사,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,5Y01,5,2제강,2제강-연주-입고,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,5W91,5,2제강,2제강-연주-입고검사,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001


In [83]:
# 피벗테이블 생성
pivot_table = query_data.pivot_table(values='LOT_NO',
                                     index=['공장구분','공정명'],
                                     aggfunc="count").reset_index()

# 특정컬럼을 기준으로 데이터 내림차순 정렬
pivot_table.reset_index(drop=True, inplace=True)

# 결과 확인
pivot_table

,공장구분,공정명,LOT_NO
0,1단조,단조-1800톤 프레스,24024
1,1단조,단조-단조검사,24024
2,1단조,단조-대형단조임가공,708
3,1단조,단조-연마기,6
4,1단조,단조-예열,206
...,...,...,...
242,특수제강,특가-절단외주,35
243,특수제강,특가-중간검사,16
244,특수제강,특가-포장,425
245,특수제강,특가-합병투입,2


In [84]:
# pivot_table.to_excel('process_code.xlsx', sheet_name='process_code', index=False)

In [86]:
process_code = pd.read_excel('process_code_new.xlsx', sheet_name='Sheet1')
process_code

,공정명,공정명_new
0,단조-1800톤 프레스,1단조_1800톤 프레스
1,단조-단조검사,1단조_단조검사
2,단조-대형단조임가공,1단조_대형단조임가공
3,단조-연마기,1단조_연마
4,단조-예열,1단조_예열
...,...,...
242,특가-절단외주,특수제강_절단
243,특가-중간검사,특수제강_중간검사
244,특가-포장,특수제강_포장
245,특가-합병투입,특수제강_합병투입


In [87]:
process_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   공정명      247 non-null    object
 1   공정명_new  247 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [88]:
# query_data와 process_code 데이터프레임을 '공정명'기준으로 inner merge
query_data = pd.merge(query_data, process_code, how='inner', on='공정명')

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('투입정보_형상')
query_data.insert(position,'공정명_new',
                  query_data.pop('공정명_new'))

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,1,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001


In [89]:
# 컬럼별 null 데이터 갯수 확인
column_null_counts = query_data.isnull().sum(axis=0)

# 결과 확인
print(column_null_counts)

LOT_NO        0
사내강종          0
대강종명          0
대강종코드         0
대중강종코드        0
대중소강종코드       0
사내강종명         0
품종            0
품종명           0
주문형상          0
열처리           0
표면            0
설계외경          0
설계폭           0
최초투입형상        0
HEAT_NO       0
HEAT_NO_구분    0
재료코드          0
재료코드_구분       0
소재_외경         0
소재_폭          0
공정순위          0
공정코드          0
공정구분          0
공장구분          0
공정명           0
공정명_new       0
투입정보_형상       0
투입정보_외경       0
투입정보_폭        0
제소구분          0
생산의뢰번호_대분류    0
생산의뢰번호        0
수주년월          0
표준품목코드        0
설계조건코드        0
dtype: int64


In [90]:
# 중복된 row 데이터 삭제
query_data.drop_duplicates(inplace=True)

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,1,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001


In [91]:
# 특정컬럼을 기준으로 데이터 내림차순 정렬
query_data = query_data.sort_values(by=['LOT_NO', '공정순위'], ascending=True)
query_data.reset_index(drop=True, inplace=True)

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,1,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001


In [ ]:
query_data.info()

In [92]:
# 피벗테이블 생성
pivot_table = query_data.pivot_table(values='LOT_NO',
                                     index=['품종명',
                                            '대강종명',
                                            '대강종코드',
                                            '대중강종코드',
                                            '대중소강종코드',
                                            '사내강종명',
                                            '최초투입형상',
                                            '재료코드_구분',
                                            '재료코드',
                                            '표준품목코드',
                                            '설계조건코드',
                                            '주문형상',
                                            '설계외경',
                                            '설계폭',
                                            '열처리',
                                            '표면'],
                                     aggfunc="count").reset_index()

# 특정컬럼을 기준으로 데이터 내림차순 정렬
pivot_table = pivot_table.sort_values(by='LOT_NO', ascending=False)
pivot_table.reset_index(drop=True, inplace=True)

# 결과 확인
pivot_table

,품종명,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,최초투입형상,재료코드_구분,재료코드,표준품목코드,설계조건코드,주문형상,설계외경,설계폭,열처리,표면,LOT_NO
0,선재,스테인리스강,S,SS,SSB,STS302S1,BT,7,7730,WSSB002U5WRPKD1Y,TSS0394001,WR,5.5,5.5,SL,PK,28498
1,선재,스테인리스강,S,SS,SSB,STS304H1S5,BT,7,7730,WSSB036U4WRPKD1Y,TSS0395001,WR,5.5,5.5,SL,PK,21126
2,선재,스테인리스강,S,SS,SSB,STS304H1S5,BT,7,7730,WSSB036U3WRPKD1Y,TSS1191001,WR,5.5,5.5,SL,PK,20156
3,선재,스테인리스강,S,SS,SSB,STS302SK,BT,7,7730,WSSB045U1WRPKD1Y,TSS0395001,WR,5.5,5.5,SL,PK,19126
4,선재,스테인리스강,S,SS,SSB,STS302S1,BT,7,7730,WSSB002U3WRPKD1Y,TSS1458001,WR,5.5,5.5,SL,PK,16929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19696,반제품,스테인리스강,S,SS,SSR,321HDS1,BM,P,P240,ZSSR011ASBMBSF1N,BC00000003,BM,390.0,470.0,NH,BS,2
19697,반제품,스테인리스강,S,SS,SSR,321HDS1,BM,P,P265,ZSSR011ASBMBSF1N,BC00000003,BM,390.0,470.0,NH,BS,2
19698,반제품,스테인리스강,S,SS,SSR,321HDS1,BM,P,P290,ZSSR011ASBMBSF1N,BC00000003,BM,390.0,470.0,NH,BS,2
19699,반제품,스테인리스강,S,SS,SSR,321HDS1,BM,P,P550,ZSSR011ASBMBSF1N,BC00000003,BM,390.0,470.0,NH,BS,2


In [93]:
# 'LOT_NO' 그룹별로 알파벳 라벨링하여 새로운 파생변수 생성
query_data['공정순위_new'] = query_data.groupby('LOT_NO').cumcount().apply(lambda x: f"a{x+1:02d}")

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('공정코드')
query_data.insert(position, '공정순위_new', query_data.pop('공정순위_new'))

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정순위_new,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,a01,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,1,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,a02,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,a03,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,a04,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,a05,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,2,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,2,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001


In [94]:
# 특정컬럼의 고유값 확인
unique_values = query_data['제소구분'].unique()

# 결과 확인
unique_values

array(['1', '2'], dtype=object)

In [95]:
# 조건에 맞는 row데이터 선택하여 새로운 파생변수에 대체할 문자로 채워넣기
query_data.loc[query_data['제소구분'] == '1', '제소구분명'] = '소재공정'
query_data.loc[query_data['제소구분'] == '2', '제소구분명'] = '제품공정'

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('생산의뢰번호_대분류')
query_data.insert(position, '제소구분명', query_data.pop('제소구분명'))

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정순위_new,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분,제소구분명,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,a01,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,1,소재공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,a02,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,a03,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,a04,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,a05,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001


In [96]:
# 특정컬럼의 고유값 확인
unique_values = query_data['제소구분명'].unique()

# 결과 확인
unique_values

array(['소재공정', '제품공정'], dtype=object)

In [97]:
# 2개 이상의 특정컬럼의 문자열을 병합하여 새로운 파생변수 생성
query_data['제소구분_공정명_형상_외경_폭'] = query_data['제소구분명'] + '_' + query_data['공정명'] + '_' + query_data['투입정보_형상'] + '_' + query_data['투입정보_외경'].astype(str) + '_' + query_data['투입정보_폭'].astype(str)

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('제소구분')
query_data.insert(position,
                  '제소구분_공정명_형상_외경_폭',
                  query_data.pop('제소구분_공정명_형상_외경_폭'))

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정순위_new,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분_공정명_형상_외경_폭,제소구분,제소구분명,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,a01,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,소재공정_선재-연속가열로_BT_170.0_170.0,1,소재공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,a02,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,제품공정_선재-압연공통_BT_170.0_170.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,a03,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,제품공정_선재-선재압연검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,a04,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,제품공정_소형정정-연속어닐링로 1호_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,a05,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,제품공정_소형정정-열처리검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001


In [98]:
# 2개 이상의 특정컬럼의 문자열을 병합하여 새로운 파생변수 생성
query_data['제소구분_NEW공정명_형상_외경_폭'] = query_data['제소구분명'] + '_' + query_data['공정명_new'] + '_' + query_data['투입정보_형상'] + '_' + query_data['투입정보_외경'].astype(str) + '_' + query_data['투입정보_폭'].astype(str)

# 대상 파생당컬럼은 원래컬럼 다음에 배치
position = query_data.columns.get_loc('제소구분')
query_data.insert(position,
                  '제소구분_NEW공정명_형상_외경_폭',
                  query_data.pop('제소구분_NEW공정명_형상_외경_폭'))

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정순위_new,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분_공정명_형상_외경_폭,제소구분_NEW공정명_형상_외경_폭,제소구분,제소구분명,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,a01,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,소재공정_선재-연속가열로_BT_170.0_170.0,소재공정_소형압연_연속가열로_BT_170.0_170.0,1,소재공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,a02,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,제품공정_선재-압연공통_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,a03,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,제품공정_선재-선재압연검사_RB_42.0_42.0,제품공정_소형압연_선재압연검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,a04,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,제품공정_소형정정-연속어닐링로 1호_RB_42.0_42.0,제품공정_소형정정_연속어닐링로_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,a05,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,제품공정_소형정정-열처리검사_RB_42.0_42.0,제품공정_소형정정_열처리검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,제품공정_2제강_연주 입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,제품공정_2제강_연주 입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,제품공정_2제강_연주 입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,제품공정_2제강_연주 입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001


In [99]:
# 2개 이상의 특정컬럼의 문자열을 병합하여 새로운 파생변수 생성
query_data['설계KEY'] = query_data['표준품목코드'] + '_' + query_data['설계조건코드']

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정순위_new,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분_공정명_형상_외경_폭,제소구분_NEW공정명_형상_외경_폭,제소구분,제소구분명,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드,설계KEY
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,a01,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,소재공정_선재-연속가열로_BT_170.0_170.0,소재공정_소형압연_연속가열로_BT_170.0_170.0,1,소재공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,a02,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,제품공정_선재-압연공통_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,a03,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,제품공정_선재-선재압연검사_RB_42.0_42.0,제품공정_소형압연_선재압연검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,a04,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,제품공정_소형정정-연속어닐링로 1호_RB_42.0_42.0,제품공정_소형정정_연속어닐링로_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,a05,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,제품공정_소형정정-열처리검사_RB_42.0_42.0,제품공정_소형정정_열처리검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,제품공정_2제강_연주 입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,제품공정_2제강_연주 입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,제품공정_2제강_연주 입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,제품공정_2제강_연주 입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001


In [100]:
# 1_1차 피벗테이블 생성(제소구분_공정명_형상_외경_폭 기준)
pivot_table = query_data.pivot_table(values='제소구분_공정명_형상_외경_폭',
                                     index=['품종명',
                                            '대강종명',
                                            '대강종코드',
                                            '대중강종코드',
                                            '대중소강종코드',
                                            '사내강종명',
                                            '최초투입형상',
                                            '재료코드_구분',
                                            '재료코드',
                                            '설계KEY',
                                            '주문형상',
                                            '설계외경',
                                            '설계폭',
                                            '열처리',
                                            '표면',
                                            'LOT_NO'],
                                     columns='공정순위_new',
                                     aggfunc="sum").reset_index()

# 전체 컬럼에서 null 데이터를 None으로 변환
pivot_table = pivot_table.fillna('None')

# 결과 확인
pivot_table

공정순위_new,품종명,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,최초투입형상,재료코드_구분,재료코드,설계KEY,주문형상,설계외경,설계폭,열처리,표면,LOT_NO,a01,a02,a03,a04,a05,a06,a07,a08,a09,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20,a21,a22,a23,a24,a25,a26,a27,a28,a29,a30,a31,a32,a33,a34,a35,a36,a37,a38,a39,a40,a41,a42,a43,a44,a45,a46,a47,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57,a58,a59,a60,a61
0,대형재,공구강,T,TD,TDA,SMATF,IG,E,E030,BTDA068CAFBBSY1Y_IC00000005,FB,16.0,410.0,SLA,BS,B190374431,소재공정_3제강-배치어닐링로1호_IG_355.0_355.0,소재공정_특가-대형절단기(1080B/S)_IG_355.0_355.0,소재공정_대형정정-쇼트브라스트기_IG_355.0_355.0,소재공정_특가-대형절단기(1080B/S)_IG_355.0_355.0,소재공정_3제강-특제-정련공통_IG_355.0_355.0,소재공정_가공-비파괴검사임가공(UT)_RB_500.0_500.0,소재공정_특가-대형절단기(1080B/S)_RB_500.0_500.0,소재공정_단조-1800톤 프레스_RB_500.0_500.0,소재공정_단조-단조검사_SL_250.0_480.0,소재공정_RFM-열처리공통_SL_250.0_480.0,소재공정_분괴-연마 4호_SL_250.0_480.0,소재공정_단조-1800톤 프레스_SL_250.0_480.0,소재공정_단조-단조검사_SL_150.0_480.0,소재공정_RFM-열처리공통_SL_150.0_480.0,소재공정_분괴-연마 4호_SL_150.0_480.0,제품공정_분괴-압연_SL_150.0_480.0,제품공정_분괴-평강 압연기_FB_16.0_410.0,제품공정_분괴-압연_FB_50.0_430.0,제품공정_분괴-평강 압연기_FB_20.0_418.0,제품공정_분괴-분괴압연검사_FB_27.0_418.0,제품공정_소형정정-Bump교정기_FB_27.0_418.0,제품공정_소형정정-평강교정기_FB_27.0_418.0,제품공정_가공-배치고용화로 2호_FB_27.0_418.0,제품공정_소형정정-Bump교정기_FB_27.0_418.0,제품공정_소형정정-평강교정기_FB_27.0_418.0,제품공정_가공-배치템퍼링로 3호_FB_27.0_418.0,제품공정_소형정정-Bump교정기_FB_27.0_418.0,제품공정_소형정정-평강교정기_FB_27.0_418.0,제품공정_소형정정-황삭외주_FB_27.0_418.0,제품공정_소형정정-중간검사_FB_16.0_410.0,제품공정_소형정정-비파괴검사임가공(UT)_FB_16.0_410.0,제품공정_소형정정-대형절단기700 2호_FB_16.0_410.0,제품공정_소형정정-평강입고검사_FB_16.0_410.0,제품공정_소형정정-평강포장_FB_16.0_410.0,제품공정_소형정정-입고_FB_16.0_410.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,대형재,공구강,T,TD,TDA,SMATF,IG,E,E030,BTDA068CAFBBSY1Y_IC00000005,FB,16.0,410.0,SLA,BS,B250530409,소재공정_2제강-보기어닐링로 1호_IG_355.0_355.0,소재공정_특가-대형절단기(1080B/S)_IG_355.0_355.0,소재공정_대형정정-쇼트브라스트기_IG_355.0_355.0,소재공정_특가-대형절단기(1080B/S)_IG_355.0_355.0,소재공정_3제강-특제-정련공통_IG_355.0_355.0,소재공정_가공-배치어닐링로 1호_RB_500.0_500.0,소재공정_가공-비파괴검사임가공(UT)_RB_500.0_500.0,소재공정_단조-1800톤 프레스_RB_500.0_500.0,소재공정_단조-단조검사_SL_150.0_500.0,소재공정_RFM-열처리공통_SL_150.0_500.0,소재공정_분괴-연마 4호_SL_150.0_500.0,소재공정_가공-비파괴검사임가공(UT)_SL_150.0_500.0,소재공정_특가-대형절단기(1080B/S)_SL_150.0_500.0,제품공정_분괴-압연_SL_150.0_500.0,제품공정_분괴-평강 압연기_FB_16.0_410.0,제품공정_분괴-분괴압연검사_FB_16.0_410.0,제품공정_소형정정-Bump교정기_FB_16.0_410.0,제품공정_가공-배치고용화로 1호_FB_16.0_410.0,제품공정_소형정정-Bump교정기_FB_16.0_410.0,제품공정_소형정정-평강교정기_FB_16.0_410.0,제품공정_가공-배치템퍼링로 3호_FB_16.0_410.0,제품공정_소형정정-Bump교정기_FB_16.0_410.0,제품공정_소형정정-평강교정기_FB_16.0_410.0,제품공정_소형정정-중간검사_FB_16.0_410.0,제품공정_소형정정-비파괴검사임가공(UT)_FB_16.0_410.0,제품공정_소형정정-대형절단기1080_FB_16.0_410.0,제품공정_소형정정-중간검사_FB_16.0_410.0,제품공정_소형정정-평강입고검사_FB_16.0_410.0,제품공정_소형정정-평강포장_FB_16.0_410.0,제품공정_소형정정-입고_FB_16.0_410.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,대형재,공구강,T,TD,TDA,SMATF,IG,E,E030,BTDA068CAFBBSY1Y_IC00000005,FB,16.0,410.0,SLA,BS,B350403004,소재공정_2제강-보기어닐링로 1호_IG_355.0_355.0,소재공정_가공-대형절단기430 1호_IG_355.0_355.0,소재공정_대형정정-쇼트브라스트기_IG_355.0_355.0,소재공정_특가-대형절단기(1080B/S)_IG_355.0_355.0,소재공정_3제강-특제-정련공통_IG_355.0_355.0,소재공정_가공-배치어닐링로 3호_RB_500.0_500.0,소재공정_단조-1800톤 프레스_RB_500.0_500.0,소재공정_단조-단조검사_SL_150.0_500.0,소재공정_분괴-연마 4호_SL_150.0_500.0,소재공정_가공-비파괴검사임가공(UT)_SL_150.0_500.0,제품공정_분괴-압연_SL_150.0_500.0,제품공정_분괴-평강 압연기_FB_16.0_410.0,제품공정_분괴-분괴압연검사_FB_16.0_410.0,제품공정_소형정정-Bump교정기_FB_16.0_410.0,제품공정_가공-배치고용화로 2호_FB_16.0_410.0,제품공정_소형정정-Bump교정기_FB_16.0_410.0,제품공정_소형정정-평강교정기_FB_16.0_410.0,제품공정_가공-배치템퍼링로 1호_FB_16.0_410.0,제품공정_소형정정-Bump교정기_FB_16.0_410.0,제품공정_소형정정-평강교정기_FB_16.0_410.0,제품공정_소형정정-중간검사_FB_16.0_410.0,제품공정_소형정정-비파괴검사임가공(UT)_FB_16.0_410.0,제품공정_소형정정-대형절단기700 1호_FB_16.0_410.0,제품공정_소형정정-평강입고검사_FB_16.0_410.0,제품공정_소형정정-평강포장_FB_16.0_410.0,제품공정_소형정정-입고_FB_16.0_410.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,대형재,공구강,T,TD,TDA,SMATF,IG,E,E030,B

In [101]:
# 데이터프레임의 컬럼 이름 추출
column_names = pivot_table.columns.tolist()

# 특정 컬럼명 삭제 (예: 'LOT_NO')
if 'LOT_NO' in column_names:
    column_names.remove('LOT_NO')

column_names

['품종명',
 '대강종명',
 '대강종코드',
 '대중강종코드',
 '대중소강종코드',
 '사내강종명',
 '최초투입형상',
 '재료코드_구분',
 '재료코드',
 '설계KEY',
 '주문형상',
 '설계외경',
 '설계폭',
 '열처리',
 '표면',
 'a01',
 'a02',
 'a03',
 'a04',
 'a05',
 'a06',
 'a07',
 'a08',
 'a09',
 'a10',
 'a11',
 'a12',
 'a13',
 'a14',
 'a15',
 'a16',
 'a17',
 'a18',
 'a19',
 'a20',
 'a21',
 'a22',
 'a23',
 'a24',
 'a25',
 'a26',
 'a27',
 'a28',
 'a29',
 'a30',
 'a31',
 'a32',
 'a33',
 'a34',
 'a35',
 'a36',
 'a37',
 'a38',
 'a39',
 'a40',
 'a41',
 'a42',
 'a43',
 'a44',
 'a45',
 'a46',
 'a47',
 'a48',
 'a49',
 'a50',
 'a51',
 'a52',
 'a53',
 'a54',
 'a55',
 'a56',
 'a57',
 'a58',
 'a59',
 'a60',
 'a61']

In [102]:
# 1_2차 피벗테이블 생성
pivot_table_process = pivot_table.pivot_table(values='LOT_NO',
                                              index=column_names,
                                              aggfunc="count").reset_index()

# 특정컬럼을 기준으로 데이터 내림차순 정렬
pivot_table_process = pivot_table_process.sort_values(by='LOT_NO', ascending=False)

# 결과 확인
pivot_table_process

공정순위_new,품종명,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,최초투입형상,재료코드_구분,재료코드,설계KEY,주문형상,설계외경,설계폭,열처리,표면,a01,a02,a03,a04,a05,a06,a07,a08,a09,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20,a21,a22,a23,a24,a25,a26,a27,a28,a29,a30,a31,a32,a33,a34,a35,a36,a37,a38,a39,a40,a41,a42,a43,a44,a45,a46,a47,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57,a58,a59,a60,a61,LOT_NO
20203,선재,스테인리스강,S,SS,SSB,STS302S1,BT,7,7730,WSSB002U5WRPKD1Y_TSS0394001,WR,5.5,5.5,SL,PK,소재공정_선재-연속가열로_BT_170.0_170.0,제품공정_선재-압연공통_BT_170.0_170.0,제품공정_선재-DST_WR_5.5_5.5,제품공정_선재-선재압연검사_WR_5.5_5.5,제품공정_코일-산세_WR_5.5_5.5,제품공정_코일-입고검사_WR_5.5_5.5,제품공정_코일-포장_WR_5.5_5.5,제품공정_코일-입고_WR_5.5_5.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2088
20287,선재,스테인리스강,S,SS,SSB,STS302SK,BT,7,7730,WSSB045U1WRPKD1Y_TSS0395001,WR,5.5,5.5,SL,PK,소재공정_선재-연속가열로_BT_170.0_170.0,제품공정_선재-압연공통_BT_170.0_170.0,제품공정_선재-DST_WR_5.5_5.5,제품공정_선재-선재압연검사_WR_5.5_5.5,제품공정_코일-산세_WR_5.5_5.5,제품공정_코일-입고검사_WR_5.5_5.5,제품공정_코일-포장_WR_5.5_5.5,제품공정_코일-입고_WR_5.5_5.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1704
20403,선재,스테인리스강,S,SS,SSB,STS304H1S5,BT,7,7730,WSSB036U3WRPKD1Y_TSS1191001,WR,5.5,5.5,SL,PK,소재공정_선재-연속가열로_BT_170.0_170.0,제품공정_선재-압연공통_BT_170.0_170.0,제품공정_선재-DST_WR_5.5_5.5,제품공정_선재-선재압연검사_WR_5.5_5.5,제품공정_코일-산세_WR_5.5_5.5,제품공정_코일-입고검사_WR_5.5_5.5,제품공정_코일-포장_WR_5.5_5.5,제품공정_코일-입고_WR_5.5_5.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1691
20421,선재,스테인리스강,S,SS,SSB,STS304H1S5,BT,7,7730,WSSB036U4WRPKD1Y_TSS0395001,WR,5.5,5.5,SL,PK,소재공정_선재-연속가열로_BT_170.0_170.0,제품공정_선재-압연공통_BT_170.0_170.0,제품공정_선재-DST_WR_5.5_5.5,제품공정_선재-선재압연검사_WR_5.5_5.5,제품공정_코일-산세_WR_5.5_5.5,제품공정_코일-입고검사_WR_5.5_5.5,제품공정_코일-포장_WR_5.5_5.5,제품공정_코일-입고_WR_5.5_5.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1649
24825,선재,탄소합금강,A,AA,AAA,SWRCH10A1,BT,7,7895,WAAA387U1WRBSQ1N_TSS6216001,WR,10.0,10.0,NH,BS,소재공정_선재-연마 1호_BT_170.0_170.0,소재공정_선재-연속가열로_BT_170.0_170.0,제품공정_선재-압연공통_BT_170.0_170.0,제품공정_선재-입고검사_WR_10.0_10.0,제품공정_선재-입고_WR_10.0_10.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14516,대형재,스테인리스강,S,SS,SSV,STS630H50S1,IG,T,T080,BSSV00400RBRTY1Y_BS21464001,RB,190.0,190.0,SLA,RT,소재공정_단조-1800톤 프레스_IG_1030.0_702.0,소재공정_단조-단조검사_SB_350.0_350.0,소재공정_RFM-열처리공통_SB_350.0_350.0,소재공정_분괴-연마 7호_SB_350.0_350.0,제품공정_분괴-압연_SB_350.0_350.0,제품공정_분괴-분괴압연검사_RB_200.0_200.0,제품공정_가공-배치고용화로 1호_RB_200.0_200.0,제품공정_특가-Bump교정기_RB_200.0_200.0,제품공정_가공-배치어닐링로 3호_RB_200.0_200.0,제품공정_특가-Bump교정기_RB_200.0_200.0,제품공정_가공-대형절단기430 1호_RB_200.0_200.0,제품공정_가공-황삭기350_RB_200.0_200.0,제품공정_가공-ET/UT320_RB_190.0_190.0,제품공정_가공-중간검사_RB_190.0_190.0,제품공정_가공-대형절단기430 2호_RB_190.0_190.0,제품공정_가공-입고검사A_RB_190.0_190.0,제품공정_가공-포장A_RB_19

In [103]:
# xlsx 파일명 설정
excel_filename = f"240604_process_pivot_table.xlsx"

# pivot_table_process을 'pivot_table_process_old_공정명' 시트에 내보내기
pivot_table_process.to_excel(excel_filename, sheet_name='pivot_table_process_old_공정명', index=False)

In [104]:
# 2_1차 피벗테이블 생성(제소구분_NEW공정명_형상_외경_폭 기준)
pivot_table = query_data.pivot_table(values='제소구분_NEW공정명_형상_외경_폭',
                                     index=['품종명',
                                            '대강종명',
                                            '대강종코드',
                                            '대중강종코드',
                                            '대중소강종코드',
                                            '사내강종명',
                                            '최초투입형상',
                                            '재료코드_구분',
                                            '재료코드',
                                            '설계KEY',
                                            '주문형상',
                                            '설계외경',
                                            '설계폭',
                                            '열처리',
                                            '표면',
                                            'LOT_NO'],
                                     columns='공정순위_new',
                                     aggfunc="sum").reset_index()

# 전체 컬럼에서 null 데이터를 None으로 변환
pivot_table = pivot_table.fillna('None')

# 결과 확인
pivot_table

공정순위_new,품종명,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,최초투입형상,재료코드_구분,재료코드,설계KEY,주문형상,설계외경,설계폭,열처리,표면,LOT_NO,a01,a02,a03,a04,a05,a06,a07,a08,a09,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20,a21,a22,a23,a24,a25,a26,a27,a28,a29,a30,a31,a32,a33,a34,a35,a36,a37,a38,a39,a40,a41,a42,a43,a44,a45,a46,a47,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57,a58,a59,a60,a61
0,대형재,공구강,T,TD,TDA,SMATF,IG,E,E030,BTDA068CAFBBSY1Y_IC00000005,FB,16.0,410.0,SLA,BS,B190374431,소재공정_3제강_열처리_IG_355.0_355.0,소재공정_특수제강_절단_IG_355.0_355.0,소재공정_대형정정_쇼트브라스트기_IG_355.0_355.0,소재공정_특수제강_절단_IG_355.0_355.0,소재공정_특수제강_정련공통_IG_355.0_355.0,소재공정_가공_NDT검사_RB_500.0_500.0,소재공정_특수제강_절단_RB_500.0_500.0,소재공정_1단조_1800톤 프레스_RB_500.0_500.0,소재공정_1단조_단조검사_SL_250.0_480.0,소재공정_RFM_열처리_SL_250.0_480.0,소재공정_대형압연_연마_SL_250.0_480.0,소재공정_1단조_1800톤 프레스_SL_250.0_480.0,소재공정_1단조_단조검사_SL_150.0_480.0,소재공정_RFM_열처리_SL_150.0_480.0,소재공정_대형압연_연마_SL_150.0_480.0,제품공정_대형압연_분괴압연_SL_150.0_480.0,제품공정_대형압연_평강압연_FB_16.0_410.0,제품공정_대형압연_분괴압연_FB_50.0_430.0,제품공정_대형압연_평강압연_FB_20.0_418.0,제품공정_대형압연_분괴압연검사_FB_27.0_418.0,제품공정_소형정정_BUMP교정_FB_27.0_418.0,제품공정_소형정정_평강교정기_FB_27.0_418.0,제품공정_가공_배치고용화로_FB_27.0_418.0,제품공정_소형정정_BUMP교정_FB_27.0_418.0,제품공정_소형정정_평강교정기_FB_27.0_418.0,제품공정_가공_배치템퍼링로_FB_27.0_418.0,제품공정_소형정정_BUMP교정_FB_27.0_418.0,제품공정_소형정정_평강교정기_FB_27.0_418.0,제품공정_소형정정_황삭_FB_27.0_418.0,제품공정_소형정정_중간검사_FB_16.0_410.0,제품공정_소형정정_NDT검사_FB_16.0_410.0,제품공정_소형정정_절단_FB_16.0_410.0,제품공정_소형정정_평강입고검사_FB_16.0_410.0,제품공정_소형정정_평강포장_FB_16.0_410.0,제품공정_소형정정_입고_FB_16.0_410.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,대형재,공구강,T,TD,TDA,SMATF,IG,E,E030,BTDA068CAFBBSY1Y_IC00000005,FB,16.0,410.0,SLA,BS,B250530409,소재공정_2제강_열처리_IG_355.0_355.0,소재공정_특수제강_절단_IG_355.0_355.0,소재공정_대형정정_쇼트브라스트기_IG_355.0_355.0,소재공정_특수제강_절단_IG_355.0_355.0,소재공정_특수제강_정련공통_IG_355.0_355.0,소재공정_가공_배치어닐링로_RB_500.0_500.0,소재공정_가공_NDT검사_RB_500.0_500.0,소재공정_1단조_1800톤 프레스_RB_500.0_500.0,소재공정_1단조_단조검사_SL_150.0_500.0,소재공정_RFM_열처리_SL_150.0_500.0,소재공정_대형압연_연마_SL_150.0_500.0,소재공정_가공_NDT검사_SL_150.0_500.0,소재공정_특수제강_절단_SL_150.0_500.0,제품공정_대형압연_분괴압연_SL_150.0_500.0,제품공정_대형압연_평강압연_FB_16.0_410.0,제품공정_대형압연_분괴압연검사_FB_16.0_410.0,제품공정_소형정정_BUMP교정_FB_16.0_410.0,제품공정_가공_배치고용화로_FB_16.0_410.0,제품공정_소형정정_BUMP교정_FB_16.0_410.0,제품공정_소형정정_평강교정기_FB_16.0_410.0,제품공정_가공_배치템퍼링로_FB_16.0_410.0,제품공정_소형정정_BUMP교정_FB_16.0_410.0,제품공정_소형정정_평강교정기_FB_16.0_410.0,제품공정_소형정정_중간검사_FB_16.0_410.0,제품공정_소형정정_NDT검사_FB_16.0_410.0,제품공정_소형정정_절단_FB_16.0_410.0,제품공정_소형정정_중간검사_FB_16.0_410.0,제품공정_소형정정_평강입고검사_FB_16.0_410.0,제품공정_소형정정_평강포장_FB_16.0_410.0,제품공정_소형정정_입고_FB_16.0_410.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,대형재,공구강,T,TD,TDA,SMATF,IG,E,E030,BTDA068CAFBBSY1Y_IC00000005,FB,16.0,410.0,SLA,BS,B350403004,소재공정_2제강_열처리_IG_355.0_355.0,소재공정_가공_절단_IG_355.0_355.0,소재공정_대형정정_쇼트브라스트기_IG_355.0_355.0,소재공정_특수제강_절단_IG_355.0_355.0,소재공정_특수제강_정련공통_IG_355.0_355.0,소재공정_가공_배치어닐링로_RB_500.0_500.0,소재공정_1단조_1800톤 프레스_RB_500.0_500.0,소재공정_1단조_단조검사_SL_150.0_500.0,소재공정_대형압연_연마_SL_150.0_500.0,소재공정_가공_NDT검사_SL_150.0_500.0,제품공정_대형압연_분괴압연_SL_150.0_500.0,제품공정_대형압연_평강압연_FB_16.0_410.0,제품공정_대형압연_분괴압연검사_FB_16.0_410.0,제품공정_소형정정_BUMP교정_FB_16.0_410.0,제품공정_가공_배치고용화로_FB_16.0_410.0,제품공정_소형정정_BUMP교정_FB_16.0_410.0,제품공정_소형정정_평강교정기_FB_16.0_410.0,제품공정_가공_배치템퍼링로_FB_16.0_410.0,제품공정_소형정정_BUMP교정_FB_16.0_410.0,제품공정_소형정정_평강교정기_FB_16.0_410.0,제품공정_소형정정_중간검사_FB_16.0_410.0,제품공정_소형정정_NDT검사_FB_16.0_410.0,제품공정_소형정정_절단_FB_16.0_410.0,제품공정_소형정정_평강입고검사_FB_16.0_410.0,제품공정_소형정정_평강포장_FB_16.0_410.0,제품공정_소형정정_입고_FB_16.0_410.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,대형재,공구강,T,TD,TDA,SMATF,IG,E,E030,BTDA068CAFBBSY1Y_IC00000005,FB,16.0,410.0,SLA,BS,B350403006,소재공정_2제강_열처리_IG_355.0_355.0,소재공정_가공_절단_IG_355.0_355.0,소재공정_대형정정_쇼트브라스트기_IG_355.0_355.0,소재공정_특수제강_절단_IG_355.0_355.0,소재공정_특수제강_정련공통_IG

In [105]:
# 데이터프레임의 컬럼 이름 추출
column_names = pivot_table.columns.tolist()

# 특정 컬럼명 삭제 (예: 'LOT_NO')
if 'LOT_NO' in column_names:
    column_names.remove('LOT_NO')

column_names

['품종명',
 '대강종명',
 '대강종코드',
 '대중강종코드',
 '대중소강종코드',
 '사내강종명',
 '최초투입형상',
 '재료코드_구분',
 '재료코드',
 '설계KEY',
 '주문형상',
 '설계외경',
 '설계폭',
 '열처리',
 '표면',
 'a01',
 'a02',
 'a03',
 'a04',
 'a05',
 'a06',
 'a07',
 'a08',
 'a09',
 'a10',
 'a11',
 'a12',
 'a13',
 'a14',
 'a15',
 'a16',
 'a17',
 'a18',
 'a19',
 'a20',
 'a21',
 'a22',
 'a23',
 'a24',
 'a25',
 'a26',
 'a27',
 'a28',
 'a29',
 'a30',
 'a31',
 'a32',
 'a33',
 'a34',
 'a35',
 'a36',
 'a37',
 'a38',
 'a39',
 'a40',
 'a41',
 'a42',
 'a43',
 'a44',
 'a45',
 'a46',
 'a47',
 'a48',
 'a49',
 'a50',
 'a51',
 'a52',
 'a53',
 'a54',
 'a55',
 'a56',
 'a57',
 'a58',
 'a59',
 'a60',
 'a61']

In [106]:
# 2_2차 피벗테이블 생성
pivot_table_process = pivot_table.pivot_table(values='LOT_NO',
                                              index=column_names,
                                              aggfunc="count").reset_index()

# 특정컬럼을 기준으로 데이터 내림차순 정렬
pivot_table_process = pivot_table_process.sort_values(by='LOT_NO', ascending=False)

# 결과 확인
pivot_table_process

공정순위_new,품종명,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,최초투입형상,재료코드_구분,재료코드,설계KEY,주문형상,설계외경,설계폭,열처리,표면,a01,a02,a03,a04,a05,a06,a07,a08,a09,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20,a21,a22,a23,a24,a25,a26,a27,a28,a29,a30,a31,a32,a33,a34,a35,a36,a37,a38,a39,a40,a41,a42,a43,a44,a45,a46,a47,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57,a58,a59,a60,a61,LOT_NO
15876,선재,스테인리스강,S,SS,SSB,STS302S1,BT,7,7730,WSSB002U5WRPKD1Y_TSS0394001,WR,5.5,5.5,SL,PK,소재공정_소형압연_연속가열로_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,제품공정_소형압연_DST열처리_WR_5.5_5.5,제품공정_소형압연_선재압연검사_WR_5.5_5.5,제품공정_산세공장_산세_WR_5.5_5.5,제품공정_산세공장_입고검사_WR_5.5_5.5,제품공정_산세공장_포장_WR_5.5_5.5,제품공정_산세공장_입고_WR_5.5_5.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3178
16044,선재,스테인리스강,S,SS,SSB,STS304H1S5,BT,7,7730,WSSB036U4WRPKD1Y_TSS0395001,WR,5.5,5.5,SL,PK,소재공정_소형압연_연속가열로_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,제품공정_소형압연_DST열처리_WR_5.5_5.5,제품공정_소형압연_선재압연검사_WR_5.5_5.5,제품공정_산세공장_산세_WR_5.5_5.5,제품공정_산세공장_입고검사_WR_5.5_5.5,제품공정_산세공장_포장_WR_5.5_5.5,제품공정_산세공장_입고_WR_5.5_5.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2424
16031,선재,스테인리스강,S,SS,SSB,STS304H1S5,BT,7,7730,WSSB036U3WRPKD1Y_TSS1191001,WR,5.5,5.5,SL,PK,소재공정_소형압연_연속가열로_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,제품공정_소형압연_DST열처리_WR_5.5_5.5,제품공정_소형압연_선재압연검사_WR_5.5_5.5,제품공정_산세공장_산세_WR_5.5_5.5,제품공정_산세공장_입고검사_WR_5.5_5.5,제품공정_산세공장_포장_WR_5.5_5.5,제품공정_산세공장_입고_WR_5.5_5.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2308
15944,선재,스테인리스강,S,SS,SSB,STS302SK,BT,7,7730,WSSB045U1WRPKD1Y_TSS0395001,WR,5.5,5.5,SL,PK,소재공정_소형압연_연속가열로_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,제품공정_소형압연_DST열처리_WR_5.5_5.5,제품공정_소형압연_선재압연검사_WR_5.5_5.5,제품공정_산세공장_산세_WR_5.5_5.5,제품공정_산세공장_입고검사_WR_5.5_5.5,제품공정_산세공장_포장_WR_5.5_5.5,제품공정_산세공장_입고_WR_5.5_5.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2186
15848,선재,스테인리스강,S,SS,SSB,STS302S1,BT,7,7730,WSSB002U3WRPKD1Y_TSS1458001,WR,5.5,5.5,SL,PK,소재공정_소형압연_연속가열로_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,제품공정_소형압연_DST열처리_WR_5.5_5.5,제품공정_소형압연_선재압연검사_WR_5.5_5.5,제품공정_산세공장_산세_WR_5.5_5.5,제품공정_산세공장_입고검사_WR_5.5_5.5,제품공정_산세공장_포장_WR_5.5_5.5,제품공정_산세공장_입고_WR_5.5_5.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12656,선재,스테인리스강,S,SM,SMA,STS410S5,BT,7,7730,WSMA083UJWRPKD1Y_TSS0394002,WR,5.5,5.5,FA,PK,소재공정_소형정정_NDT검사_BT_170.0_170.0,소재공정_소형압연_연속가열로_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,제품공정_소형압연_선재압연검사_WR_5.5_5.5,제품공정_산세공장_배치분위기로_WR_5.5_5.5,제품공정_산세공장_쇼트브라스트기_WR_5.5_5.5,제품공정_산세공장_연속어닐링로_WR_5.5_5.5,제품공정_산세공장_쇼트브라스트기_WR_5.5_5.5,제품공정_산세공장_산세_WR_5.5_5.5,제품공정_산세공장_방청_WR_5.5_5.5,제품공정_산세공장_입고검사_WR_5.5_5.5,제품공정_산세공장_포장_WR_5.5_5.5,제품공정_산세공장_입고_WR_5.5_5.5,None,None,None,None,None,N

In [107]:
# pivot_table_process를 'pivot_table_process_new_공정명' 시트에 내보내기 (기존 파일에 추가)
with pd.ExcelWriter(excel_filename, mode='a', engine='openpyxl') as writer:
    pivot_table_process.to_excel(writer, sheet_name='pivot_table_process_new_공정명', index=False)

In [108]:
# process_code를 'process_code_old_new' 시트에 내보내기 (기존 파일에 추가)
with pd.ExcelWriter(excel_filename, mode='a', engine='openpyxl') as writer:
    process_code.to_excel(writer, sheet_name='process_code_old_new', index=False)

In [109]:
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정순위_new,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분_공정명_형상_외경_폭,제소구분_NEW공정명_형상_외경_폭,제소구분,제소구분명,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드,설계KEY
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,a01,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,소재공정_선재-연속가열로_BT_170.0_170.0,소재공정_소형압연_연속가열로_BT_170.0_170.0,1,소재공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,a02,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,제품공정_선재-압연공통_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,a03,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,제품공정_선재-선재압연검사_RB_42.0_42.0,제품공정_소형압연_선재압연검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,a04,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,제품공정_소형정정-연속어닐링로 1호_RB_42.0_42.0,제품공정_소형정정_연속어닐링로_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,a05,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,제품공정_소형정정-열처리검사_RB_42.0_42.0,제품공정_소형정정_열처리검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,제품공정_2제강_연주 입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,제품공정_2제강_연주 입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,제품공정_2제강_연주 입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,제품공정_2제강_연주 입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001


In [110]:
# 새로운 파생변수 생성
query_data['설계조건코드_구분'] = query_data['설계조건코드'].str.slice(0, 1)

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정순위_new,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분_공정명_형상_외경_폭,제소구분_NEW공정명_형상_외경_폭,제소구분,제소구분명,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드,설계KEY,설계조건코드_구분
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,a01,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,소재공정_선재-연속가열로_BT_170.0_170.0,소재공정_소형압연_연속가열로_BT_170.0_170.0,1,소재공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001,T
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,a02,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,제품공정_선재-압연공통_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001,T
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,a03,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,제품공정_선재-선재압연검사_RB_42.0_42.0,제품공정_소형압연_선재압연검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001,T
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,a04,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,제품공정_소형정정-연속어닐링로 1호_RB_42.0_42.0,제품공정_소형정정_연속어닐링로_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001,T
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,a05,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,제품공정_소형정정-열처리검사_RB_42.0_42.0,제품공정_소형정정_열처리검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,제품공정_2제강_연주 입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001,T
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,제품공정_2제강_연주 입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001,T
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,제품공정_2제강_연주 입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001,T
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,제품공정_2제강_연주 입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001,T


In [111]:
# 특정컬럼의 고유값 확인
unique_values = query_data['설계조건코드_구분'].unique()

# 결과 확인
unique_values

array(['T', 'B', 'I'], dtype=object)

In [112]:
# 조건에 맞는 row데이터 선택하여 새로운 파생변수에 대체할 문자로 채워넣기
query_data.loc[query_data['설계조건코드_구분'] == 'T', '설계조건코드_구분'] = 'BT'
query_data.loc[query_data['설계조건코드_구분'] == 'B', '설계조건코드_구분'] = 'BM'
query_data.loc[query_data['설계조건코드_구분'] == 'I', '설계조건코드_구분'] = 'IG'

# 결과 확인
query_data

,LOT_NO,사내강종,대강종명,대강종코드,대중강종코드,대중소강종코드,사내강종명,품종,품종명,주문형상,열처리,표면,설계외경,설계폭,최초투입형상,HEAT_NO,HEAT_NO_구분,재료코드,재료코드_구분,소재_외경,소재_폭,공정순위,공정순위_new,공정코드,공정구분,공장구분,공정명,공정명_new,투입정보_형상,투입정보_외경,투입정보_폭,제소구분_공정명_형상_외경_폭,제소구분_NEW공정명_형상_외경_폭,제소구분,제소구분명,생산의뢰번호_대분류,생산의뢰번호,수주년월,표준품목코드,설계조건코드,설계KEY,설계조건코드_구분
0,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,10,a01,8601,8,소형압연,선재-연속가열로,소형압연_연속가열로,BT,170.0,170.0,소재공정_선재-연속가열로_BT_170.0_170.0,소재공정_소형압연_연속가열로_BT_170.0_170.0,1,소재공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001,BT
1,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,20,a02,8100,8,소형압연,선재-압연공통,소형압연_압연공통,BT,170.0,170.0,제품공정_선재-압연공통_BT_170.0_170.0,제품공정_소형압연_압연공통_BT_170.0_170.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001,BT
2,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,30,a03,8W31,8,소형압연,선재-선재압연검사,소형압연_선재압연검사,RB,42.0,42.0,제품공정_선재-선재압연검사_RB_42.0_42.0,제품공정_소형압연_선재압연검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001,BT
3,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,40,a04,B201,B,소형정정,소형정정-연속어닐링로 1호,소형정정_연속어닐링로,RB,42.0,42.0,제품공정_소형정정-연속어닐링로 1호_RB_42.0_42.0,제품공정_소형정정_연속어닐링로_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001,BT
4,A780556800,AAA329,탄소합금강,A,AA,AAA,AE1016,S,소형재,RB,FA,PM,40.5,40.5,BT,S30191,S,7895,7,170.0,170.0,50,a05,BW41,B,소형정정,소형정정-열처리검사,소형정정_열처리검사,RB,42.0,42.0,제품공정_소형정정-열처리검사_RB_42.0_42.0,제품공정_소형정정_열처리검사_RB_42.0_42.0,2,제품공정,D,D23S02W0067,202302,SAAA32900RBPMY1Y,TSS0256001,SAAA32900RBPMY1Y_TSS0256001,BT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482983,B450481300,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,제품공정_2제강_연주 입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001,BT
2482984,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,제품공정_2제강_연주 입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001,BT
2482985,B450481400,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,20,a02,5Y01,5,2제강,2제강-연주-입고,2제강_연주 입고,BT,160.0,160.0,제품공정_2제강-연주-입고_BT_160.0_160.0,제품공정_2제강_연주 입고_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001,BT
2482986,B450481500,CCE165,탄소합금강,C,CC,CCE,SCM420HDS1,Z,반제품,BT,NH,BS,160.0,160.0,BT,S57766,S,6200,6,160.0,160.0,10,a01,5W91,5,2제강,2제강-연주-입고검사,2제강_연주 입고검사,BT,160.0,160.0,제품공정_2제강-연주-입고검사_BT_160.0_160.0,제품공정_2제강_연주 입고검사_BT_160.0_160.0,2,제품공정,D,D24Z05W0047,202405,ZCCE165U1BTBSR1N,TS25808001,ZCCE165U1BTBSR1N_TS25808001,BT


In [113]:
# 특정컬럼의 고유값 확인
unique_values = query_data['설계조건코드_구분'].unique()

# 결과 확인
unique_values

array(['BT', 'BM', 'IG'], dtype=object)

In [114]:
# 특정조건을 만족하는 row데이터 추출
filtered_data = query_data[query_data['최초투입형상'] != query_data['설계조건코드_구분']]

# 특정조건을 만족하는 row데이터의 특정컬럼의 고유값 추출
unique_values = filtered_data['LOT_NO'].unique()

# 결과 확인
unique_values

array(['A810563006', 'A810563021', 'A810563022', ..., 'B450024031',
       'B450024032', 'B450024033'], dtype=object)

In [115]:
# unique_values의 갯수 확인
# 길이 구하기
length = len(unique_values)

# 결과 확인
print(length)

2390
